# Data Visualisation

# Data Preparation

In [ ]:
# Change directory to keep paths consistent
%cd /Users/brk/projects/masters/SU/ergo/src

## Imports and setup

In [ ]:
# minimise me
%load_ext autoreload
%autoreload 2
import seaborn as sns
import seaborn.objects as so
import matplotlib.pyplot as plt
import matplotlib as mpl
import ipywidgets as widgets
import datetime
from ipywidgets import interact, interactive, fixed, interact_manual
import pandas as pd
import numpy as np
import models
import vis
import common
import read
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
import sklearn
import tqdm
import logging as l
import tqdm
import yaml
import glob
from matplotlib.colors import LogNorm
import re
from sklearn.metrics import classification_report
from scipy.stats import f

plt.rcParams.update({
    'font.size' : 9,                   # Set font size to 11pt
    'axes.labelsize': 9,               # -> axis labels
    'legend.fontsize': 9,              # -> legends
    'font.family': 'serif',
    'text.usetex': True,
    'text.latex.preamble': (            # LaTeX preamble
        r'\usepackage{lmodern}'
    )
})

## Utility functions

In [ ]:
# minimise me
def prettify_col_name(x):
    return x.split('.')[-1].replace('_', ' ').title()

def calculate_prediction_ellipse(x, y, alpha=0.95):
    """Given some x and y data, calculate the (1-alpha) confidence ellipse."""
    data = np.column_stack((x, y)) # Combine x and y into a single data array
    num_dimensions = data.shape[1]
    num_data_points = data.shape[0]
    # Estimate the sample covariance matrix
    sample_covariance_matrix = np.cov(data, rowvar=False)
    # Calculate the sample mean for each dimension
    sample_mean = np.mean(data, axis=0)
    # Generate angles for the ellipse
    theta = np.linspace(0, 2*np.pi, num=100)
    # Calculate the radius of the ellipse. `f.ppf` is the inverse of the CDF
    radius = np.sqrt(
        num_dimensions * (num_data_points - 1) / (num_data_points - num_dimensions) *
        (1 + 1/num_data_points) * f.ppf(1 - alpha, num_dimensions, num_data_points - num_dimensions)
    )
#     print(sample_covariance_matrix)
    # Compute the Cholesky decomposition of the covariance matrix
    chol_cov_matrix = np.linalg.cholesky(sample_covariance_matrix)
    # Generate ellipse offset based on Cholesky decomposition
    ellipse_offset = np.outer(np.cos(theta), chol_cov_matrix[0, :]) + np.outer(np.sin(theta), chol_cov_matrix[1, :])
    # Calculate the points of the prediction interval ellipse
    prediction_ellipse_points = sample_mean + radius * ellipse_offset
    return prediction_ellipse_points

def get_npz_data_from_model(model_dir):
    """Given a directory of a model, return it's y_pred and y_true."""
    data = np.load(f'{model_dir}/y_val_true_y_val_pred.npz')
    y_true = data['y_true']
    y_pred = data['y_pred']
    return y_true, y_pred

def show_conf_mat_from_model(model_dir, ax=None):
    """Given a directory of a model, plot its confidence matrix"""
    y_true, y_pred = get_npz_data_from_model(model_dir)
    cm_val = tf.math.confusion_matrix(
        y_true.flatten(), 
        y_pred.flatten()
    ).numpy()
    p = vis.conf_mat(cm_val / cm_val.sum(axis=0), ax=ax)
    return p

## Load data

In [ ]:
# Read in data from hpar optimisation
paths = sorted(glob.glob('../saved_models/results_*_optuna.jsonl'))
print(f'Reading data from\n', "\n".join(paths))
dfs = map(
    lambda path: pd.read_json(path, lines=True),
    paths
)
# Concat the dataframes together, and then do a 
# copy to avoid a dataframe fragmentation warning
# Reset the index to avoid a seaborn error https://github.com/mwaskom/seaborn/issues/3291
df = pd.concat(dfs).reset_index(drop=True).copy()
df['preprocessing.num_gesture_classes'] = df['preprocessing.num_gesture_classes'].fillna('51')
df['preprocessing.num_gesture_classes'] = df['preprocessing.num_gesture_classes'].astype(int).astype(str)

# 50-class HFFNNs don't make sense, remove them
df = df[~(
    (df['model_type'] == 'HFFNN')
    & (df['preprocessing.num_gesture_classes'] == '50')
)]

df.groupby(['model_type', 'preprocessing.num_gesture_classes']).size()

## Constants

In [ ]:
model_colours = {
    'FFNN': 'tab:blue',
    'HFFNN': 'tab:orange',
    'CuSUM': 'tab:green',
    'HMM': 'tab:red',
    'SVM': 'tab:purple',
}
palette = 'Spectral'
other_colours = [
    'tab:brown',
    'tab:pink',
    'tab:grey',
    'tab:olive',
    'tab:cyan',
]
WIDTH = 5.5

rename_hpars = {
    'ffnn.dropout_rate'            : 'Dropout Rate',
    'ffnn.l2_coefficient.log10'    : 'L2 ($\log_{10}$)',
    'nn.batch_size.log10'          : 'Batch Size ($\log_{10}$)',
    'nn.learning_rate.log10'       : 'LR ($\log_{10}$)',
    'ffnn.l2_coefficient'          : 'L2 Coef.',
    'nn.batch_size'                : 'Batch Size',
    'nn.learning_rate'             : 'LR',
    'ffnn.num_layers'              : '\#Layers',
    'ffnn.nodes_per_layer.1'       : '\#Nodes (layer 1)',
    'ffnn.nodes_per_layer.2'       : '\#Nodes (layer 2)',
    'ffnn.nodes_per_layer.3'       : '\#Nodes (layer 3)',
    'ffnn.nodes_per_layer.-1'      : '\#Nodes (last layer)',
    'ffnn.nodes_per_layer.1.log10' : '\#Nodes (layer 1, $\log_{10}$)',
    'ffnn.nodes_per_layer.2.log10' : '\#Nodes (layer 2, $\log_{10}$)',
    'ffnn.nodes_per_layer.3.log10' : '\#Nodes (layer 3, $\log_{10}$)',
    'ffnn.nodes_per_layer.-1.log10': '\#Nodes (last layer, $\log_{10}$)',
    'val.macro avg.f1-score'       : '$F_1$-score',
    'val.macro avg.recall'         : 'Recall',
    'val.macro avg.precision'      : 'Precision',
    'val.loss.log10'               : 'Val. Loss ($\log_{10}$)',
    'trn.loss.log10'               : 'Trn. Loss ($\log_{10}$)',
    'val.loss'                     : 'Val. Loss',
    'trn.loss'                     : 'Trn. Loss',
}
# Add rename cols for the HFFNNs
rename_hpars |= {
    f'hffnn.majority.{k}': f'Maj. {v}' for k, v in rename_hpars.items() if 'nn.' in k
} | {
    f'hffnn.minority.{k}': f'Min. {v}' for k, v in rename_hpars.items() if 'nn.' in k
}

## Calculate some auxillary values

In [ ]:
# Preprocess the data a little bit, and get a list of dependant variables
# Preprocess the df a bit to get some nice-to-use columns

# Convert nodes_per_layer into nodes_per_layer.1, .2, .3
prefixes = (
    'ffnn.nodes_per_layer',
    'hffnn.majority.ffnn.nodes_per_layer',
    'hffnn.minority.ffnn.nodes_per_layer',
)
for prefix in prefixes:
    for i in (1, 2, 3):
        df[f'{prefix}.{i}'] = df[prefix].apply(
            lambda x: x[i-1] if isinstance(x, list) and len(x) >= i else None
        )
    df[f'{prefix}.-1'] = df[prefix].apply(
        lambda x: x[-1] if isinstance(x, list) and len(x) >= 1 else None
    )


# Calculate ratios
avgs = ('macro avg', 'weighted avg')
metrics = ('f1-score', 'precision', 'recall')

for avg in avgs:
    for metric in metrics:
        df[f'ratio.{avg}.{metric}'] = df[f'trn.{avg}.{metric}'] / df[f'val.{avg}.{metric}']
        df[f'ratio.{avg}.{metric}'] = np.where(
            np.isfinite(df[f'ratio.{avg}.{metric}']),
            df[f'ratio.{avg}.{metric}'],
            np.nan
        )
df['ratio.loss'] = df['trn.loss'] / df['val.loss']

# Print out a list of dependant variables
# dep_vars = sorted([
#     c for c in df.columns 
#     if 'val' not in c and 'trn' not in c and 'ratio' not in c and c not in (
#         'saved_at', 'fit_time', 'preprocessing.gesture_allowlist', 
# )], key=lambda c: str(c))
# print(f"Dependant variables: {dep_vars}")
# print("\nVariables which change:")
# max_len = max(map(lambda x: len(x), dep_vars))
# Print out all dependant variables that change
# for var in dep_vars:
#     uniq = df[var].apply(lambda x: str(x) if isinstance(x, list) else x).unique()
#     if len(uniq) > 1:
#         print(f"{var: <{max_len}} {uniq}")
        
df['ffnn.dropout_rate'] = np.round(df['ffnn.dropout_rate'], 6)

df['val.pred_time_per_obs'] = df['val.pred_time'] / df['val.num_observations']
df['trn.pred_time_per_obs'] = df['trn.pred_time'] / df['trn.num_observations']
df['fit_time_per_obs'] = df['fit_time'] / df['trn.num_observations']


# Add some log10 columns
log10_cols = [
    'val.loss',
    'trn.loss',
    'ffnn.l2_coefficient',
    'nn.batch_size',
    'nn.learning_rate',
    'ffnn.nodes_per_layer.1',
    'ffnn.nodes_per_layer.2',
    'ffnn.nodes_per_layer.3',
    'ffnn.nodes_per_layer.-1',
    'hffnn.majority.ffnn.nodes_per_layer.1',
    'hffnn.minority.ffnn.nodes_per_layer.1',
    'hffnn.majority.ffnn.nodes_per_layer.2',
    'hffnn.minority.ffnn.nodes_per_layer.2',
    'hffnn.majority.ffnn.nodes_per_layer.3',
    'hffnn.minority.ffnn.nodes_per_layer.3',
    'hffnn.majority.ffnn.nodes_per_layer.-1',
    'hffnn.minority.ffnn.nodes_per_layer.-1',
    'hffnn.majority.ffnn.l2_coefficient',
    'hffnn.minority.ffnn.l2_coefficient',
    'hffnn.majority.nn.batch_size',
    'hffnn.minority.nn.batch_size',
    'hffnn.majority.nn.learning_rate',
    'hffnn.minority.nn.learning_rate',
]

df[[f'{c}.log10' for c in log10_cols]] = np.log10(df[log10_cols])
# df[[f'{c}.log10' for c in log10_cols]] = df[[f'{c}.log10' for c in log10_cols]].fillna(0)

for c in log10_cols:
    if df[f'{c}.log10'].isna().any():
        print(f"{df[f'{c}.log10'].isna().sum()} NaNs in {c}.log10")

# There are a *lot* of columns. Here's a more-useful subset
subset_cols = [
    c for c in df.columns
    if (not re.search(r'((trn|val)\.\d+\.)|weighted avg', c)) and 
        (c not in [
            'hmm', 'lstm', 'ffnn', 'nn', 'hffnn', 'cusum', 'svm',
            'preprocessing.n_timesteps',
            'preprocessing.gesture_allowlist',
            'preprocessing.gesture_allowlist',
        ])
]

df['ffnn.num_layers'] = df['ffnn.nodes_per_layer'].apply(
    lambda x: np.nan if type(x) is not list else len(x)
)
df['hffnn.majority.ffnn.num_layers'] = df['hffnn.majority.ffnn.nodes_per_layer'].apply(
    lambda x: np.nan if type(x) is not list else len(x)
)
df['hffnn.minority.ffnn.num_layers'] = df['hffnn.minority.ffnn.nodes_per_layer'].apply(
    lambda x: np.nan if type(x) is not list else len(x)
)

In [ ]:
df[
    (df['model_type'] == "FFNN")
    & (df['preprocessing.num_gesture_classes'] == '51')
].groupby('ffnn.num_layers').size()

In [ ]:
%%script false --no-raise-error
fig, axs = plt.subplots(3, 1, figsize=(WIDTH, WIDTH))
ngestures = ('51', '50', '5')
xmin = None
xmax = None
for ax, ngesture in zip(axs, ngestures):
    sns.scatterplot(
        data=df[df['preprocessing.num_gesture_classes'] == ngesture],
        x='saved_at',
        y='preprocessing.seed',
        hue='model_type',
        s=10,
        hue_order=list(model_colours.keys()),
    #     alpha=0.1,
        ax=ax,
    edgecolor=None,
    )
    ax.set_title(f'{ngesture} gestures')
#     if xmin is None: xmin = ax.get_xlim()[0]
#     if xmax is None: xmax = ax.get_xlim()[1]
#     xmin = min(xmin, ax.get_xlim()[0])
#     xmax = min(xmax, ax.get_xlim()[1])
# for ax in axs:
#     ax.set_xlim((xmin, xmax))
plt.tight_layout()

print(df.shape)
df = df[
    (df['model_type'] != 'HFFNN')
    | (df['saved_at'] > pd.to_datetime('2023-10-01T11:00:00'))
]
print(df.shape)

# Plotting

## Bar plot of number of observations

In [ ]:
(
    X_trn, X_val, y_trn, y_val, dt_trn, dt_val
) = common.read_and_split_from_npz("../gesture_data/trn_20_10.npz")

fig, axs = plt.subplots(1, 2, figsize=(WIDTH, WIDTH*.5))
counts = pd.Series(y_trn).value_counts()
axs[0].bar(counts.index, counts)


counts = pd.Series(y_trn[y_trn != 50]).value_counts()
axs[1].bar(counts.index, counts)

axs[0].set(
    title='Number of classes\n',
    xlabel='Class',
    ylabel='Count',
)
axs[1].set(
    title='Number of classes\n(excluding class 50)',
    xlabel='Class',
    ylabel='Count',
)
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_class_imbalance.pdf',
    bbox_inches='tight'
)
plt.show()

## Precision vs Recall vs $F_1$

### 51 classes, Precision vs Recall vs $F_1$

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(WIDTH, WIDTH*0.5))

recall_grid, precision_grid = np.meshgrid(
    np.linspace(0, 1, 100), 
    np.linspace(0, 1, 100)
)
f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

contours = axs[0].contour(
    recall_grid, 
    precision_grid,
    f1_score, 
    levels=np.linspace(0.1, 1, 10), 
    colors='black',
    alpha=0.25
)
axs[0].clabel(contours, inline=True, fontsize=8, fmt='%.2f')

sns.scatterplot(
    data=df[df['preprocessing.num_gesture_classes'] == '51'],
    x='val.macro avg.precision',
    y='val.macro avg.recall',
    s=5,
    alpha=0.5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    ax=axs[0],
    edgecolor=None,
    legend=False,
)
axs[0].set_xlim((-0.1, 1.1))
axs[0].set_ylim((-0.1, 1.1))
axs[0].plot([0,1], [0,1], color='black', alpha=.1)
axs[0].set_title(f'Precision vs Recall\n51-classes')
axs[0].set_xlabel(f'Precision')
axs[0].set_ylabel(f'Recall')
# axs[0].legend().set_title("Model ")


order = ['FFNN', 'SVM', 'HFFNN', 'HMM', 'CuSUM']
sns.stripplot(
    data=df[df['preprocessing.num_gesture_classes'] == '51'],
    y='val.macro avg.f1-score',
    x='model_type',
    s=2,
    alpha=0.5,
#     order=list(model_colours.keys()),
    order=order,
    hue='model_type',
    hue_order=list(model_colours.keys()),
#     hue_order=order,
    ax=axs[1],
    legend=False,
)
axs[1].set_title(f'$F_1$-score\n51-classes')
axs[1].set_xlabel(f'Model Type')
axs[1].set_ylabel(f'$F_1$-score')
axs[1].set_ylim((-0.1, 1.1))
axs[1].grid(axis='y')
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_precision_recall_51_classes.pdf', 
    bbox_inches='tight'
)


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH))

data = df.loc[
    df['preprocessing.num_gesture_classes'] == '51',
    ['model_type', 'val.macro avg.recall', 'val.macro avg.precision']
].melt(
    id_vars=['model_type'], 
    var_name='metric', 
    value_name='value'
)
data['metric'] = data['metric'].replace({
    'val.macro avg.recall': 'Recall',
    'val.macro avg.precision': 'Precision',
})

sns.stripplot(
    data=data,
    x="model_type", 
    y="value", 
    hue="metric",
    order=list(model_colours.keys()),
    palette=palette,
    dodge=True,
    alpha=0.75,
    size=2,
    ax=ax
)
ax.set_ylim((-0.05, 1.05))

ax.set_title(f'Precision and recall for all model types')
ax.set_xlabel(f'Model Type')
ax.set_ylabel(f'Metric Value')
ax.legend().set_title("Metric")
ax.set_yticks(np.arange(0, 1.1, .1))
ax.set_yticklabels(np.round(np.arange(0, 1.1, .1), 1))
ax.grid(axis='y')

plt.savefig(
    '../../report/src/imgs/graphs/05_precision_recall_stripplot.pdf', 
    bbox_inches='tight'
)
plt.tight_layout()

### Precision vs Recall for all models

In [ ]:
# Three plots showing the precision and recall for all models.
# Each plot showing either 51, 50, or 5 gesture classes
fig, axs = plt.subplots(1, 3, figsize=(WIDTH, WIDTH/3.))
n_gesture_classes = ('51', '50', '5')

recall_grid, precision_grid = np.meshgrid(
    np.linspace(0, 1, 100), 
    np.linspace(0, 1, 100)
)
f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

for ax, n_classes in zip(axs, n_gesture_classes):
    contours = ax.contour(
        recall_grid, 
        precision_grid,
        f1_score, 
        levels=np.linspace(0.1, 1, 5), 
        colors='black',
        alpha=0.25
    )
    ax.clabel(contours, inline=True, fontsize=8, fmt='%.2f')
    
    sns.scatterplot(
        data=df[df['preprocessing.num_gesture_classes'] == n_classes],
        x='val.macro avg.precision',
        y='val.macro avg.recall',
        s=2.5,
        alpha=0.5,
        hue='model_type',
        hue_order=list(model_colours.keys()),
        ax=ax,
        legend=False,
        edgecolor=None,
    )
    ax.set_xlim((-0.1, 1.1))
    ax.set_ylim((-0.1, 1.1))
    ax.set_xlabel('Precision')
    ax.set_ylabel('Recall')
    ax.plot([0,1], [0,1], color='black', alpha=.1)
    ax.set_title(f'{n_classes} gesture classes')
#     ax.legend().set_title('Model Type')

plt.tight_layout()

In [ ]:
# Three plots showing the precision and recall for all models.
# Each plot showing either 51, 50, or 5 gesture classes
# n_gesture_classes = ('51', '50', '5')

recall_grid, precision_grid = np.meshgrid(
    np.linspace(0, 1, 100), 
    np.linspace(0, 1, 100)
)
f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

for model_type in model_colours.keys():
    fig, ax = plt.subplots(1, 1, figsize=(WIDTH*0.5, WIDTH*0.5))
    
    contours = ax.contour(
        recall_grid, 
        precision_grid,
        f1_score, 
        levels=np.linspace(0.1, 1, 10), 
        colors='black',
        alpha=0.25
    )
    ax.clabel(contours, inline=True, fontsize=8, fmt='%.2f')
    
    sns.scatterplot(
        data=df[df['model_type'] == model_type],
        x='val.macro avg.precision',
        y='val.macro avg.recall',
        s=10,
#         alpha=0.5,
        hue='preprocessing.num_gesture_classes',
        style='preprocessing.num_gesture_classes',
        hue_order=['5', '50', '51'],
        style_order=['5', '50', '51'],
        ax=ax,
    edgecolor=None,
    )
    ax.set_xlim((-0.05, 1.05))
    ax.set_ylim((-0.05, 1.05))
    ax.set_xlabel('Precision')
    ax.set_ylabel('Recall')
    ax.plot([0,1], [0,1], color='black', alpha=.1)
    ax.set_title(f'{model_type} $F_1$-score by number of classes')
    ax.legend().set_title('Number of classes')
    
    plt.savefig(
        f'../../report/src/imgs/graphs/05_f1_by_num_gesture_classes_{model_type.lower()}.pdf',
        bbox_inches='tight',
    )

plt.tight_layout()

## Best model by highest lower 90th percentile

In [ ]:
from scipy import stats
type_to_hpars = {
    'CuSUM': ['cusum.thresh'],
    'HMM': ['hmm.covariance_type'],
    'FFNN': [
        'ffnn.dropout_rate',
        'ffnn.l2_coefficient',
        'ffnn.nodes_per_layer.1',
        'ffnn.nodes_per_layer.2',
        'ffnn.nodes_per_layer.3',
        'nn.batch_size',
        'nn.learning_rate',
    ],
    'HFFNN': [
        'hffnn.majority.ffnn.dropout_rate',
        'hffnn.majority.ffnn.l2_coefficient',
        'hffnn.majority.ffnn.nodes_per_layer.1',
        'hffnn.majority.ffnn.nodes_per_layer.2',
        'hffnn.majority.ffnn.nodes_per_layer.3',
        'hffnn.majority.nn.epochs',
        'hffnn.majority.nn.batch_size',
        'hffnn.majority.nn.learning_rate',
        'hffnn.minority.ffnn.dropout_rate',
        'hffnn.minority.ffnn.l2_coefficient',
        'hffnn.minority.ffnn.nodes_per_layer.1',
        'hffnn.minority.ffnn.nodes_per_layer.2',
        'hffnn.minority.ffnn.nodes_per_layer.3',
        'hffnn.minority.nn.epochs',
        'hffnn.minority.nn.batch_size',
        'hffnn.minority.nn.learning_rate',
    ],
    'SVM': ['svm.c', 'svm.class_weight'],
}

def tenth_conf_interval(series):
    mean = np.mean(series)
    sem = stats.sem(series)
    if sem == 0:
        return mean
    confidence_interval = stats.t.interval(
        0.90, 
        len(series) - 1, 
        loc=mean, 
        scale=sem
    )
#     print(mean, sem, confidence_interval)
    return confidence_interval[0]


all_hpars = ['model_type'] + [
    item 
    for sublist in list(type_to_hpars.values()) 
    for item in sublist
]

subset = df[(df['preprocessing.num_gesture_classes'] == '51')]
# print(subset.shape)
gb = subset.groupby(all_hpars, dropna=False)
subset['val.macro avg.f1-score.count'] = (
    gb['val.macro avg.f1-score']
    .transform('count')
)

subset = subset[
    subset['val.macro avg.f1-score.count'].between(5, 100)
]

subset['val.macro avg.f1-score.tenth_conf_interval'] = (
    gb['val.macro avg.f1-score']
    .transform(tenth_conf_interval)
)
subset['val.macro avg.f1-score.mean']  = gb['val.macro avg.f1-score'].transform('mean')
subset['val.macro avg.f1-score.min']   = gb['val.macro avg.f1-score'].transform('min')
subset['val.macro avg.f1-score.max']   = gb['val.macro avg.f1-score'].transform('max')
subset['val.macro avg.f1-score.std']   = gb['val.macro avg.f1-score'].transform('std')
subset['val.macro avg.f1-score.count'] = gb['val.macro avg.f1-score'].transform('count')

subset['group_idx'] = gb.ngroup()
subset = subset.sort_values('val.macro avg.f1-score.tenth_conf_interval')

subset

### Plot models grouped by hyperparameters

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH/5.))

order = subset.sort_values(
    'val.macro avg.f1-score.tenth_conf_interval'
)['group_idx'].unique()

sns.stripplot(
    data=subset,
    y='val.macro avg.f1-score',
    x='group_idx',
    hue='model_type',
    order=order,
    size=5,
    alpha=0.5,
    ax=ax,
)

sns.pointplot(
    data=subset, 
    x="group_idx", 
    y="val.macro avg.f1-score", 
    hue="model_type",
    linestyle="none", 
    errorbar=None,
    marker="_", 
    markersize=5, 
    palette='dark:black',
    markeredgewidth=1,
    zorder=10,
    ax=ax,
)

ax.set(
    title='$F_1$ score for each set of hyperparameters, by model type',
    xlabel='Hyperparameter index',
    ylabel='$F_1$-score',
    ylim=((-0.05, 1.05))
)
ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(
    order,
    rotation=90,
)

handles, labels = ax.get_legend_handles_labels()
ax.add_artist(plt.legend(handles[:-5], labels[:-5], title="Model Type"))

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH*0.5))

# data = subset[subset['val.macro avg.f1-score.tenth_conf_interval'] > 0.7]
data = subset.sort_values('val.macro avg.f1-score.tenth_conf_interval').tail(250)

order = data.sort_values(
    'val.macro avg.f1-score.tenth_conf_interval'
)['group_idx'].unique()

sns.stripplot(
    data=data,
    y='val.macro avg.f1-score',
    x='group_idx',
    hue='model_type',
    order=order,
    hue_order=list(model_colours.keys()),
    size=5,
    jitter=False,
    alpha=0.5,
    ax=ax,
)

sns.pointplot(
    data=data, 
    x="group_idx", 
    y="val.macro avg.f1-score", 
    hue="model_type",
    linestyle="none", 
    errorbar=None,
    marker="_", 
    markersize=5, 
    palette='dark:black',
    markeredgewidth=1,
    zorder=10,
    ax=ax,
)

ax.set(
    title=f'$F_1$ score for top {len(order)} hyperparameters',
    xlabel='Hyperparameter index',
    ylabel='$F_1$-score',
)
ax.yaxis.grid(True)
ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(
    order,
    rotation=90,
)

handles, labels = ax.get_legend_handles_labels()
ax.add_artist(plt.legend(handles[:5], labels[:5], title="Model Type"))

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_best_hpar_comparison.pdf',
    bbox_inches='tight',
)
plt.show()

In [ ]:
print("Best model:")
from pprint import pprint
pprint(
    subset
    .sort_values('val.macro avg.f1-score.tenth_conf_interval', ascending=False)
    .groupby('group_idx')
    .tail(1)
#     [type_to_hpars['FFNN']]
    .head(1)
    .squeeze()
    .to_dict()
)

In [ ]:
model_type = 'FFNN'

def prettify_col(col):
    return {
        'val.macro avg.f1-score.mean': '$F_1$-score Mean',
        'val.macro avg.f1-score.std': '$F_1$-score Std.Dev.',
        'group_idx': 'Index',
        'cusum.thresh': 'Threshold',
        'ffnn.dropout_rate': 'Dropout Rate',
        'ffnn.l2_coefficient': 'L2 Coefficient',
        'ffnn.l2_coefficient.log10': '$\log_{10}(\text{L2 Coefficient})$',
        'ffnn.nodes_per_layer.1': 'Nodes (layer 1)',
        'ffnn.nodes_per_layer.2': 'Nodes (layer 2)',
        'ffnn.nodes_per_layer.3': 'Nodes (layer 3)',
        'hffnn.majority.ffnn.dropout_rate': 'Majority: Dropout Rate',
        'hffnn.majority.ffnn.l2_coefficient': 'Majority: L2 Coefficient',
        'hffnn.majority.ffnn.nodes_per_layer.1': 'Majority: Nodes (layer 1)',
        'hffnn.majority.ffnn.nodes_per_layer.2': 'Majority: Nodes (layer 2)',
        'hffnn.majority.ffnn.nodes_per_layer.3': 'Majority: Nodes (layer 3)',
        'hffnn.majority.nn.batch_size': 'Majority: Batch Size',
        'hffnn.majority.nn.epochs': 'Majority: Epochs',
        'hffnn.majority.nn.learning_rate': 'Majority: Learning Rate',
        'hffnn.majority.nn.optimizer': 'Majority: Optimizer',
        'hffnn.minority.ffnn.dropout_rate': 'Minority: Dropout Rate',
        'hffnn.minority.ffnn.l2_coefficient': 'Minority: L2 Coefficient',
        'hffnn.minority.ffnn.nodes_per_layer.1': 'Minority: Nodes (layer 1)',
        'hffnn.minority.ffnn.nodes_per_layer.2': 'Minority: Nodes (layer 2)',
        'hffnn.minority.ffnn.nodes_per_layer.3': 'Minority: Nodes (layer 3)',
        'hffnn.minority.nn.batch_size': 'Minority: Batch Size',
        'hffnn.minority.nn.epochs': 'Minority: Epochs',
        'hffnn.minority.nn.learning_rate': 'Minority: Learning Rate',
        'hffnn.minority.nn.optimizer': 'Minority: Optimizer',
        'hmm.covariance_type': 'Covariance Type',
        'nn.batch_size': 'Batch Size',
        'nn.batch_size.log10': '$\log_{10}(\text{Batch Size})$',
        'nn.epochs': 'Epochs',
        'nn.learning_rate': 'Learning Rate',
        'nn.learning_rate.log10': '$\log_{10}(\text{Learning Rate})$',
        'nn.optimizer': 'Optimizer',
        'svm.c': 'C',
        'svm.class_weight': 'Class Weight',
    }.get(col, col)

def df_to_latex(df, model_type):
    path = f'../../report/src/tables/05_best_{model_type.lower().replace(" ", "_")}_hpars.generated.tex'
    print('DONT FORGET TO UPDATE LaTeX tables: ', path)
    df.to_latex(
        path,
        caption=f'Top {len(df)} performing {model_type} hyperparameter combinations, ordered by '
                f'the lower bound of the 90 percent confidence interval for $F_1$-score.',
        label=f'tab:05_best_{model_type.lower().replace(" ", "_")}_hpars',
        index=False,
        float_format=lambda x: '%.3e' % x,
        na_rep='-'
    )

for model_type in type_to_hpars.keys():
    latex_df = (
        subset[
            subset['model_type'] == model_type
        ]
        .sort_values('val.macro avg.f1-score.tenth_conf_interval', ascending=False)
        .groupby('group_idx')
        .tail(1)
        [
            ['group_idx', 'val.macro avg.f1-score.mean', 'val.macro avg.f1-score.std'] 
            + type_to_hpars[model_type]
        ]
        .head(10)
        .reset_index(drop=True)
        .rename(columns=prettify_col)
        .replace({
            'tied': 'Tied',
            'spherical': 'Spherical',
            'diag': 'Diagonal',
            'full': 'Full',
            'balanced': 'Balanced',
        } | {} if model_type != 'SVM' else {
            np.nan: 'Unbalanced',
        } | {} if model_type not in ('HFFNN', 'FFNN') else {
            np.nan: 'None',
        })
    )
    display(latex_df)
    
    if model_type == 'HFFNN':
        majority = latex_df[[c for c in latex_df.columns if 'Minority' not in c]]
        minority = latex_df[[c for c in latex_df.columns if 'Majority' not in c]]
        majority.columns = [c.replace("Majority: ", "") for c in majority.columns]
        minority.columns = [c.replace("Minority: ", "") for c in minority.columns]
        df_to_latex(majority, 'Majority HFFNN')
        df_to_latex(minority, 'Minority HFFNN')
    else:
        df_to_latex(latex_df, model_type)
    

### Model types grouped by hpar index

In [ ]:
from matplotlib.gridspec import GridSpec
fig = plt.figure(figsize=(WIDTH, WIDTH))
gs = GridSpec(4, 2, figure=fig)
ax1 = fig.add_subplot(gs[0, :])
ax2 = fig.add_subplot(gs[1, :])
ax3 = fig.add_subplot(gs[2, :])
ax4 = fig.add_subplot(gs[3, 0])
ax5 = fig.add_subplot(gs[3, 1])

model_axs = {
    'FFNN':  ax1,
    'HFFNN': ax2,
    'SVM':   ax3,
    'HMM':   ax4,
    'CuSUM': ax5,
}

for model_type, color in model_colours.items():
    ax = model_axs[model_type]
    data = subset[subset['model_type'] == model_type]
    
    order = data.sort_values(
        'val.macro avg.f1-score.tenth_conf_interval'
    )['group_idx'].unique()

    sns.stripplot(
        data=data,
        y='val.macro avg.f1-score',
        x='group_idx',
#         hue='model_type',
        color=color,
        order=order,
        size=5,
        alpha=0.5,
        ax=ax,
        jitter=False,
        legend=False,
    )

    sns.pointplot(
        data=data, 
        x="group_idx", 
        y="val.macro avg.f1-score", 
        hue="model_type",
        linestyle="none", 
        errorbar=None,
        marker="_", 
        markersize=5, 
        palette='dark:black',
        markeredgewidth=1,
        zorder=10,
        ax=ax,
        legend=False,
    )
    
    ax.set(
        title=f'$F_1$-score for each set of {model_type} hyperparameters',
        xlabel='Hyperparameter index',
        ylabel='$F_1$-score',
        ylim=((-0.05, 1.05)),
    )
    ax.yaxis.grid(True)
#     ax.set_xticks(ax.get_xticks())
#     ax.set_xticklabels(order, rotation=90)

    xticks = ax.get_xticks()
    if len(xticks) > 10:
        ax.set_xticks([xticks[i] for i in range(len(xticks)) if i % 4 == 0])

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_hpar_comparison_per_model_type.pdf',
    bbox_inches='tight',
)
plt.show()

## Get statistics for each hyperparameter combo

In [ ]:
data = df[
    (df['model_type'] == 'FFNN') & 
    (df['preprocessing.num_gesture_classes'] == '51') &
    (df['preprocessing.seed'] == 42.0)
]

data[[c for c in subset_cols if 'ffnn.' in c or 'nn.' in c]].head(31)
data[subset_cols].head(31)

## ECDF for all model types

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH))
for model_type, color in model_colours.items():
    scores = df.loc[
        (df['preprocessing.num_gesture_classes'] == '51')
        & (df['model_type'] == model_type),
        'val.macro avg.f1-score'
    ]
    sorted_scores = np.sort(scores)
    ecdf = np.arange(1, len(sorted_scores) + 1) / len(sorted_scores)
    ax.plot(sorted_scores, ecdf, color=color, label=model_type)
plt.legend()
ax.set(
    title='ECDF for all model types',
    xlabel='$F_1$-score',
    ylabel='Cumulative Probability',
)
plt.show()

## Training vs Inference times

In [ ]:
# Three plots showing the precision and recall for all models.
# Each plot showing either 51, 50, or 5 gesture classes
fig, axs = plt.subplots(1, 2, figsize=(WIDTH, WIDTH*.5))

df['val.pred_time_per_obs.log10'] = np.log10(df['val.pred_time_per_obs'])
df['trn.pred_time_per_obs.log10'] = np.log10(df['trn.pred_time_per_obs'])
df['fit_time_per_obs.log10'] = np.log10(df['fit_time_per_obs'])

sns.scatterplot(
    data=df[df['preprocessing.num_gesture_classes'] == '51'],
    x='val.pred_time_per_obs.log10',
    y='trn.pred_time_per_obs.log10',
    s=10,
#     alpha=0.5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    ax=axs[0],
    edgecolor=None,
)

sns.scatterplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
    ],
    x='fit_time_per_obs.log10',
    y='trn.pred_time_per_obs.log10',
    s=10,
#     alpha=0.5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    ax=axs[1],
    edgecolor=None,
)

axs[0].set(
    title='Validation vs training inference time',
    xlabel='Validation inference time\n($\log_{10}$ seconds/observation)',
    ylabel='Training inference time\n($\log_{10}$ seconds/observation)',
)
axs[1].set(
    title='Fitting vs inference time',
    xlabel='Fitting time\n($\log_{10}$ seconds/observation)',
    ylabel='Inference Time\n($\log_{10}$ seconds/observation)',
)

for ax in axs.flatten():
    ax.legend().set_title("Model Type")

plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_inf_trn_times_per_obs.pdf',
    bbox_inches='tight'
)

## Inference time vs $F_1$

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH))

sns.scatterplot(
    data=df[df['preprocessing.num_gesture_classes'] == '51'],
    x='val.pred_time_per_obs.log10',
    y='val.macro avg.f1-score',
    s=10,
    alpha=0.5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    ax=ax,
    edgecolor=None,
)

ax.set_ylim((-0.05, 1.05))
ax.set_ylabel('$F_1$-score')
ax.set_xlabel('Inference time\n($\log_{10}$ seconds/observation)')
ax.set_title('Inference time per observation against $F_1$ score')
ax.legend().set_title("Model Type")
plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_inference_time_per_obs_vs_f1.pdf', 
    bbox_inches='tight'
)

## Confusion Matrices of the best models

In [ ]:
ngestures = sorted(df['preprocessing.num_gesture_classes'].unique())
model_types = sorted(df['model_type'].unique())

fig, axs = plt.subplots(
    len(ngestures),
    len(model_types),
    # TODO this ratio might need to be swapped
    figsize=(WIDTH, WIDTH * len(model_types) / len(ngestures)),
    squeeze=False
)

for i, ngesture in enumerate(ngestures):
    for j, model_type in enumerate(model_types):
        best = df[
            (df['preprocessing.num_gesture_classes'] == ngesture) &
            (df['model_type'] == model_type)
        ].sort_values('val.macro avg.f1-score', ascending=False)
        if len(best) == 0:
            axs[i,j].axis('off')
            continue
        best = best.iloc[0]
        print(ngesture, model_type, best['model_dir'])
        try:
            show_conf_mat_from_model(f"../{best['model_dir']}", axs[i, j])
        except FileNotFoundError:
            show_conf_mat_from_model(f"./{best['model_dir']}", axs[i, j])
        axs[i, j].set(
            title=f"Best {model_type}: {ngesture} gestures\n($F_1=${np.round(best['val.macro avg.f1-score'], 4)})",
        )

plt.tight_layout()

## Mean Confidence Matrices

### 5 Gesture classes

In [ ]:
nclasses = '5'
data = df[
    (df['preprocessing.num_gesture_classes'] == nclasses)
]
conf_mats = {}
conf_mat_totals = {}

hpar = 'model_type'

assert len(data[hpar].unique()) < 10

for i, row in data.iterrows():
    y_true, y_pred = get_npz_data_from_model('../' + row['model_dir'])
    cm = tf.math.confusion_matrix(
        y_true.flatten(), 
        y_pred.flatten()
    ).numpy()
    f1_score = sklearn.metrics.f1_score(
        y_true.flatten(), 
        y_pred.flatten(),
        average='macro',
        zero_division=0,
    )
    hpar_item = row[hpar]
    
    if hpar_item in conf_mats:
        conf_mats[hpar_item] += cm.astype(float) * f1_score
        conf_mat_totals[hpar_item] += f1_score
    else:
        conf_mats[hpar_item] = cm.astype(float) * f1_score
        conf_mat_totals[hpar_item] = f1_score

fig, axs = plt.subplots(
    2,2,
    figsize=(WIDTH, WIDTH)
)
print(conf_mats.keys())
axs = axs.flatten()
for i, (hpar_item, conf_mat) in enumerate(conf_mats.items()):
#     conf_mat[-1, -1] = 0
#     conf_mat /= conf_mat_totals[hpar_item]
#     conf_mat /= conf_mat.max()
#     print(conf_mat_totals[hpar_item])
    # Normalize the matrix manually to get nicer annotations
    vis.conf_mat(conf_mat / conf_mat.sum(axis=0), ax=axs[i], norm=None)
    axs[i].set_title(
        f'{hpar_item} Confusion Matrices\n(weighted mean, {nclasses} classes)'
    )

plt.tight_layout()
plt.savefig(
    f'../../report/src/imgs/graphs/05_mean_conf_mat_{nclasses}_classes.pdf',
    bbox_inches='tight',
)

### 50 Gesture classes

In [ ]:
nclasses = '50'
data = df[
    (df['preprocessing.num_gesture_classes'] == nclasses)
]
conf_mats = {}
conf_mat_totals = {}

hpar = 'model_type'

assert len(data[hpar].unique()) < 10

for i, row in data.iterrows():
    y_true, y_pred = get_npz_data_from_model('../' + row['model_dir'])
    cm = tf.math.confusion_matrix(
        y_true.flatten(), 
        y_pred.flatten()
    ).numpy()
    f1_score = sklearn.metrics.f1_score(
        y_true.flatten(), 
        y_pred.flatten(),
        average='macro',
        zero_division=0,
    )
    hpar_item = row[hpar]
    
    if hpar_item in conf_mats:
        conf_mats[hpar_item] += cm.astype(float) * f1_score
        conf_mat_totals[hpar_item] += f1_score
    else:
        conf_mats[hpar_item] = cm.astype(float) * f1_score
        conf_mat_totals[hpar_item] = f1_score

fig, axs = plt.subplots(
    2, 2,
    figsize=(WIDTH, WIDTH)
)
print(conf_mats.keys())
axs = axs.flatten()
for i, (hpar_item, conf_mat) in enumerate(conf_mats.items()):
#     conf_mat /= conf_mat.max()
    vis.conf_mat(conf_mat / conf_mat.sum(axis=0), ax=axs[i], norm=None)
#     vis.conf_mat(conf_mat, ax=axs[i])
    axs[i].set_title(
        f'{hpar_item} Confusion Matrices\n(weighted mean, {nclasses} classes)'
    )

plt.tight_layout()
plt.savefig(
    f'../../report/src/imgs/graphs/05_mean_conf_mat_{nclasses}_classes.pdf',
    bbox_inches='tight',
)

### 51 Gesture classes

In [ ]:
nclasses = '51'
data = df[
    (df['preprocessing.num_gesture_classes'] == nclasses)
]
conf_mats = {}
conf_mat_totals = {}

hpar = 'model_type'

assert len(data[hpar].unique()) < 10

for i, row in data.iterrows():
    try:
        y_true, y_pred = get_npz_data_from_model('../' + row['model_dir'])
    except FileNotFoundError:
        y_true, y_pred = get_npz_data_from_model('./' + row['model_dir'])
    cm = tf.math.confusion_matrix(
        y_true.flatten(), 
        y_pred.flatten()
    ).numpy()
    f1_score = sklearn.metrics.f1_score(
        y_true.flatten(), 
        y_pred.flatten(),
        average='macro',
        zero_division=0,
    )
    hpar_item = row[hpar]
    
    if hpar_item in conf_mats:
        conf_mats[hpar_item] += cm.astype(float) * f1_score
        conf_mat_totals[hpar_item] += f1_score
    else:
        conf_mats[hpar_item] = cm.astype(float) * f1_score
        conf_mat_totals[hpar_item] = f1_score
    if i % 100 == 0:
        print(i, end=' ', flush=True)

In [ ]:
fig, axs = plt.subplots(
    3, 2,
    figsize=(WIDTH, WIDTH*3/2)
)
axs = axs.flatten()
for i, (hpar_item, conf_mat) in enumerate(conf_mats.items()):
    vis.conf_mat(conf_mat / conf_mat.sum(axis=0), ax=axs[i], norm=None)
    axs[i].set_title(
        f'{hpar_item} Confusion Matrices\n(weighted mean, {nclasses} classes)'
    )
axs[-1].axis('off')

plt.tight_layout()
plt.savefig(
    f'../../report/src/imgs/graphs/05_mean_conf_mat_{nclasses}_classes.pdf',
    bbox_inches='tight',
)

## Conf Mats for each num-gesture-class with precision-recall plot

In [ ]:
recall_grid, precision_grid = np.meshgrid(
    np.linspace(0, 1, 100), 
    np.linspace(0, 1, 100)
)
f1_score_grid = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)


for model_type in ('FFNN', 'HFFNN', 'SVM', 'CuSUM', 'HMM'):
    data = df[
        (df['model_type'] == model_type)
    ]
    conf_mats = {}
    conf_mat_totals = {}

    hpar = 'preprocessing.num_gesture_classes'

    assert len(data[hpar].unique()) < 10
    print(model_type, flush=True)

    for i, row in data.sort_values(hpar).iterrows():
        try:
            y_true, y_pred = get_npz_data_from_model('../' + row['model_dir'])
        except FileNotFoundError:
            y_true, y_pred = get_npz_data_from_model('./' + row['model_dir'])
        cm = tf.math.confusion_matrix(
            y_true.flatten(), 
            y_pred.flatten()
        ).numpy()
        f1_score = sklearn.metrics.f1_score(
            y_true.flatten(), 
            y_pred.flatten(),
            average='macro',
            zero_division=0,
        )
        hpar_item = row[hpar]

        if hpar_item in conf_mats:
            conf_mats[hpar_item] += cm.astype(float) * f1_score
            conf_mat_totals[hpar_item] += f1_score
        else:
            conf_mats[hpar_item] = cm.astype(float) * f1_score
            conf_mat_totals[hpar_item] = f1_score
        if i % 100 == 0:
            print(i, end=' ', flush=True)

    if model_type == 'HFFNN':
        fig, axs = plt.subplots(
            1, 2,
            figsize=(WIDTH, WIDTH*.5),
            squeeze=False,
        )
    else:
        fig, axs = plt.subplots(
            2, 2,
            figsize=(WIDTH, WIDTH),
            squeeze=False,
        )
    axs = axs.flatten()

    for i, (hpar_item, conf_mat) in enumerate(conf_mats.items()):
        cmat = conf_mat / conf_mat_totals[hpar_item]
        vis.conf_mat(cmat, ax=axs[i])
        axs[i].set_title(
            f'Weighted Confusion Matrix\n'
            f'{model_type} {hpar_item}-class'
        )

    contours = axs[-1].contour(
        recall_grid, 
        precision_grid,
        f1_score_grid, 
        levels=np.linspace(0.1, 1, 10), 
        colors='black',
        alpha=0.25
    )
    axs[-1].clabel(contours, inline=True, fontsize=8, fmt='%.2f')
    
    sns.scatterplot(
        data=df[df['model_type'] == model_type],
        x='val.macro avg.precision',
        y='val.macro avg.recall',
        s=5,
        alpha=0.5,
        hue='preprocessing.num_gesture_classes',
        style='preprocessing.num_gesture_classes',
        hue_order=['5', '50', '51'],
        style_order=['5', '50', '51'],
        ax=axs[-1],
        edgecolor=None,
    )
    axs[-1].set(
        xlim=(-0.05, 1.05),
        ylim=(-0.05, 1.05),
        xlabel='Precision',
        ylabel='Recall',
        title=f'{model_type} $F_1$-score by number of classes',
    )
    axs[-1].plot([0,1], [0,1], color='black', alpha=.25)
    axs[-1].legend().set_title('Classes')

    plt.tight_layout()
    plt.savefig(
        f'../../report/src/imgs/graphs/05_mean_conf_mat_{model_type.lower()}.pdf',
        bbox_inches='tight',
    )
    plt.show()

## Regularization Plots

In [ ]:
n_gesture_classes = (
    '5',
    '50',
    '51',
)
# TODO this ratio might need to be changed
fig, axs = plt.subplots(
    3, 2, 
    figsize=(WIDTH, WIDTH/3.)
)

for i, ngestures in enumerate(n_gesture_classes):
    data = df[df['preprocessing.num_gesture_classes'] == ngestures]
    sns.scatterplot(
        data=data,
        x='ffnn.l2_coefficient',
        y='ratio.macro avg.f1-score',
        ax=axs[i, 0]
    edgecolor=None,
    )
    sns.scatterplot(
        data=data,
        x='ffnn.dropout_rate',
        y='ratio.macro avg.f1-score',
        ax=axs[i, 1]
    edgecolor=None,
    )
    axs[i, 0].set_title(f'{ngestures} gestures')
    axs[i, 1].set_title(f'{ngestures} gestures')

plt.suptitle("$F_1$-ratio against regularisation")
plt.tight_layout()

## Ratio $F_1$ scores vs actual $F_1$ scores

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(WIDTH, WIDTH))


sns.scatterplot(
    data=df[df['preprocessing.num_gesture_classes'] == '51'],
    x='val.macro avg.f1-score',
    y='trn.macro avg.f1-score',
    s=5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    alpha=0.5,
    ax=axs[0, 0],
    edgecolor=None,
)
axs[0, 0].set_xlim((-0.05, 1.05))
axs[0, 0].set_ylim((-0.05, 1.05))
axs[0, 0].plot([0,1], [0,1], color='black', alpha=.1)
axs[0, 0].set_title("a) Training vs Validation $F_1$ score\n")
axs[0, 0].set_xlabel("Validation $F_1$")
axs[0, 0].set_ylabel("Training $F_1$")
axs[0, 0].legend().set_title('Model Type')

sns.scatterplot(
    data=df[df['preprocessing.num_gesture_classes'] == '51'],
    x='val.macro avg.f1-score',
    y='trn.macro avg.f1-score',
    s=5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    alpha=0.5,
    ax=axs[0, 1],
    edgecolor=None,
)
axs[0, 1].set_xlim((0.5, 1.05))
axs[0, 1].set_ylim((0.5, 1.05))
axs[0, 1].plot([0,1], [0,1], color='black', alpha=.1)
axs[0, 1].set_title("b) Training vs Validation $F_1$ score\n(magnified)")
axs[0, 1].set_xlabel("Validation $F_1$")
axs[0, 1].set_ylabel("Training $F_1$")
axs[0, 1].legend().set_title('Model Type')

sns.scatterplot(
    data=df[df['preprocessing.num_gesture_classes'] == '51'],
    x='val.macro avg.f1-score',
    y='ratio.macro avg.f1-score',
    s=5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    alpha=0.5,
    ax=axs[1, 0],
    edgecolor=None,
)
axs[1, 0].set_xlim((-0.05, 1.05))
axs[1, 0].set_title("c) $F_1$-ratio vs $F_1$-score\n")
axs[1, 0].set_xlabel("Validation $F_1$")
axs[1, 0].set_ylabel(r"$F_1$-ratio ($\frac{Training}{Validation}$)")
axs[1, 0].legend().set_title('Model Type')
axs[1, 0].plot([0,1], [1, 1], color='black', alpha=.1)

sns.scatterplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
        & (df['val.macro avg.f1-score'] >= 0.5)
    ],
    x='val.macro avg.f1-score',
    y='ratio.macro avg.f1-score',
    s=5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    alpha=0.5,
    ax=axs[1, 1],
    edgecolor=None,
)

axs[1, 1].set_xlim((0.5, 1.05))
axs[1, 1].set_title("d) $F_1$-ratio vs $F_1$-score\n(magnified)")
axs[1, 1].set_xlabel("Validation $F_1$")
axs[1, 1].set_ylabel(r"$F_1$-ratio ($\frac{Training}{Validation}$)")
axs[1, 1].legend().set_title('Model Type')
axs[1, 1].plot([0,1], [1, 1], color='black', alpha=.1)

plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_f1_vs_f1_ratio.pdf'
)


## Training/validation loss ratios

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(WIDTH, WIDTH*.5))

sns.scatterplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
        & (df['model_type'] == 'FFNN')
    ],
    y='val.loss',
    x='trn.loss',
    color='tab:blue',
    s=20,
    alpha=0.5,
    ax=axs[0],
    edgecolor=None,
)


sns.scatterplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
        & (df['model_type'] == 'FFNN')
    ].assign(**{
        'ratio.loss': lambda x: x['trn.loss'] / x['val.loss']
    }),
    x='ratio.loss',
    y='val.loss',
    color='tab:blue',
    s=20,
    alpha=0.5,
    ax=axs[1],
    edgecolor=None,
)


axs[0].set_ylim((-0.05, 3.6))
axs[1].set_ylim((-0.05, 3.6))

axs[1].plot([1, 1], [0, axs[1].get_ylim()[1]], color='black', alpha=.1)

min_max = min(axs[0].get_xlim()[1], axs[0].get_ylim()[1] )
axs[0].plot([0, min_max], [0, min_max], color='black', alpha=.1)

axs[0].set(
    title='Validation loss vs training loss\n(FFNN only)',
    xlabel='Training Loss',
    ylabel='Validation Loss',
)
axs[1].set(
    title='Validation loss vs loss ratio\n(FFNN only)',
    xlabel=r'Loss ratio ($\frac{Training}{Validation}$)',
    ylabel='Validation Loss',
)

plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_val_trn_loss_ratios.pdf'
)

print("TODO: The training and validation loss aren't comparable because the training loss is weighed but validation loss is not.")
print("TODO: also not comparable because of dropout")

## Precision vs Recall plots for each model individually

In [ ]:
for model_type, color in model_colours.items():
    data = df[
        (df['preprocessing.num_gesture_classes'] == '51')
        & (df['model_type'] == model_type)
    ]

    fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH))
    p_min = data['val.macro avg.precision'].min() / 1.10
    p_max = data['val.macro avg.precision'].max() * 1.10
    r_min = data['val.macro avg.recall'].min()    / 1.10
    r_max = data['val.macro avg.recall'].max()    * 1.10
    print(f"{p_min=}, {p_max=}, {r_min=}, {r_max=}, ")
    recall_grid, precision_grid = np.meshgrid(
        np.linspace(p_min, p_max, 100), 
        np.linspace(r_min, r_max, 100)
    )
    f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

    print(f1_score.min())
    contours = ax.contour(
        recall_grid, 
        precision_grid,
        f1_score, 
        levels=np.linspace(
            f1_score.min(),
            f1_score.max(),
            10
        ), 
        colors='black',
        alpha=0.25
    )
    ax.clabel(contours, inline=True, fontsize=8, fmt='%.2f')

    sns.scatterplot(
        data=data,
        y='val.macro avg.recall',
        x='val.macro avg.precision',
        color=color,
        alpha=0.5,
        s=20,
        ax=ax,
    edgecolor=None,
    )

    p_range = p_max - p_min
    r_range = r_max - r_min
    ax.set(
        title=f"Precision vs Recall For {model_type} models\n($F_1$ contours in grey)",
        xlabel='Precision',
        ylabel='Recall',
        xlim=(p_min - p_range*0.025, p_max + p_range*0.025),
        ylim=(r_min - r_range*0.025, r_max + r_range*0.025),
    )

    plt.savefig(
        f'../../report/src/imgs/graphs/05_in_depth_{model_type.lower()}_p_vs_r.pdf',
        bbox_inches='tight',
    )
    plt.show()

## In depth FFNN plots

In [ ]:
data = df[
    (df['preprocessing.num_gesture_classes'] == '51') 
    & (df['model_type'] == 'FFNN')
]
hyperpars = [
    'nn.learning_rate.log10',
    'nn.batch_size.log10',
    'ffnn.num_layers',
    'ffnn.nodes_per_layer.1.log10',
    'ffnn.nodes_per_layer.2.log10',
    'ffnn.nodes_per_layer.3.log10',
    'ffnn.nodes_per_layer.-1.log10',
    'ffnn.l2_coefficient.log10',
    'ffnn.dropout_rate',
]
data['Recall $>$ 0.7'] = data['val.macro avg.recall'] > 0.7

data['ffnn.nodes_per_layer.1.log10'].fillna(0, inplace=True)
data['ffnn.nodes_per_layer.2.log10'].fillna(0, inplace=True)
data['ffnn.nodes_per_layer.3.log10'].fillna(0, inplace=True)
data['ffnn.nodes_per_layer.-1.log10'].fillna(0, inplace=True)


### x=precision y=recall hue=nlayers

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(WIDTH, WIDTH*0.5))
sns.scatterplot(
    data=data,
    x='val.macro avg.precision',
    y='val.macro avg.recall',
    hue='ffnn.num_layers',
    s=10,
    edgecolor='black',
#     alpha=0.75,
    legend=False,
    palette=['tab:blue', 'tab:orange', 'tab:green'],
    ax=axs[1]
)
# vis.fmt_legend(axs[0])
vis.add_grid(axs[1])
axs[1].set(
    xticks=np.arange(0, 1.01, 0.2),
    yticks=np.arange(0, 1.01, 0.2),
)

sns.stripplot(
    data=data,
    y='val.macro avg.f1-score',
    x='ffnn.num_layers',
    hue='ffnn.num_layers',
    palette=['tab:blue', 'tab:orange', 'tab:green'],
    legend=False,
    s=2.5,
    alpha=0.5,
    ax=axs[0],
    native_scale=True,
    edgecolor='black',
)
axs[0].set(
    xlim=(0, 4),
    xticks=[1, 2, 3],
    yticks=np.arange(0, 1.01, 0.2),
)
vis.add_grid(axs[0])

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=p,y=r,h=nlayers.pdf',
    bbox_inches='tight',
)
plt.show()


### x=hpar y=recall, all hyperparameters

In [ ]:
aspect=2
g = sns.catplot(
    data=vis.add_jitter(data, hyperpars).melt( 
#         data=data[data['ffnn.num_layers'] == nlayers].melt( 
        id_vars=['Recall $>$ 0.7'], 
        value_vars=hyperpars,
    ), 
    x="value", 
    col="variable",
    col_wrap=2,
    hue="Recall $>$ 0.7",
    kind="violin",
    sharex=False,
    
    dodge=True,
    inner='stick',
    linewidth=1,
    split=True,
    
#     size=2.5,
#     jitter=0.25,
#     alpha=0.5,
    height=WIDTH*0.5 / aspect,
    legend=False,
    aspect=aspect,
)

for ax in g.axes.flat:
    hpar = ax.get_title().split(' = ')[1]
    ax.set_title(f'{rename_hpars[hpar]}')
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,c=hpar,x=hpar,y=recall>70.pdf',
    bbox_inches='tight',
)

### x=LR y=Nodes layer 1

In [ ]:
# #Nodes layer 1 vs LR has something happening
fig, ax = plt.subplots(1, 1, figsize=(0.5*WIDTH, 0.5*WIDTH))
sns.scatterplot(
    data=vis.add_jitter(data, hyperpars),
    y='ffnn.nodes_per_layer.1.log10',
    x='nn.learning_rate.log10',
    hue='val.macro avg.f1-score',
    s=10,
    edgecolor='black',
    palette='Spectral',
)
plt.gca().set(
    xticks=np.arange(-6, -1, 1),
    yticks=np.arange(0.5, 3, 0.5),
)
vis.add_grid()
vis.fmt_legend()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn_x=lr,y=npl1,h=f1.pdf',
    bbox_inches='tight'
)

In [ ]:
sns.scatterplot(
    data=vis.add_jitter(data, hyperpars, amount=0.05, clamp=False),
    x='ffnn.dropout_rate',
    y='ffnn.num_layers',
    hue='val.macro avg.f1-score',
    s=10,
#     alpha=0.75,
    edgecolor='black',
    legend=False,
    palette='Spectral',
)

### N-layer FFNNs

In [ ]:
(
    data
    .groupby(['ffnn.num_layers'])
    [['val.macro avg.f1-score', 'val.macro avg.precision', 'val.macro avg.recall']]
    .agg(['count', 'mean', 'median', 'min', 'max'])
    .T
    .round(3)
)

In [ ]:
np.sort(
    data['val.macro avg.f1-score']
    .sort_values()
    .tail(50)
    .round(3)
    .unique()
)[-2:]

#### x=hpar y=recall, all hyperparameters

In [ ]:
for nlayers in [1, 2, 3]:
    aspect=2
    subset_hpars = [
        hpar for hpar in hyperpars
        if data.loc[data['ffnn.num_layers'] == nlayers, hpar].nunique() > 1
    ]
    print([data.loc[data['ffnn.num_layers'] == nlayers, hpar].nunique() for hpar in hyperpars])
    print(subset_hpars)
    g = sns.catplot(
        data=vis.add_jitter(
            data[data['ffnn.num_layers'] == nlayers], subset_hpars
        ).melt( 
            id_vars=['Recall $>$ 0.7'], 
            value_vars=subset_hpars,
        ), 
        x="value", 
        col="variable",
        col_wrap=2,
        hue="Recall $>$ 0.7",
        kind="violin",
        sharex=False,

        dodge=True,
        inner='stick',
        linewidth=1,
        split=True,

    #     size=2.5,
    #     jitter=0.25,
    #     alpha=0.5,
        height=WIDTH*0.5 / aspect,
        legend=False,
        aspect=aspect,
    )

    for ax in g.axes.flat:
        hpar = ax.get_title().split(' = ')[1]
        ax.set_title(f'{rename_hpars[hpar]}')
    plt.tight_layout()

#### Precision recall histogram

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(2*WIDTH,2*WIDTH*.3))
for ax, nlayers in zip(axs, [1,2,3]):
    sns.histplot(
        data=data[data['ffnn.num_layers'] == nlayers],
        x='val.macro avg.precision',
        y='val.macro avg.recall',
        bins=(15, 15),
#         edgecolor='black',
#         s=10,
        cmap='Spectral',
        vmin=0,
        vmax=160,
        ax=ax,
        cbar=nlayers == 3,
    )
    vis.add_grid(ax)
#     if nlayers == 3:
#         vis.add_cbar(
#             fig, ax,
#             data=
#             vmin=0, vmax=1,
#         )
    ax.set(
        xlim=(-0.05, 1.05),
        ylim=(-0.05, 1.05),
        title=f'{nlayers}-layer FFNNs',
        ylabel=None if nlayers != 1 else rename_hpars[ax.get_ylabel()],
        xlabel=rename_hpars[ax.get_xlabel()],
    )

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=p,y=r,c=nlayers,histplot.pdf',
    bbox_inches='tight',
)
plt.show()

#### x=p,y=r,h=lr,c=nlayers

In [ ]:
# Three clusters in top right of precision-recall plots 
# -> caused by learning rate which needs to be _just_ right.
fig, axs = plt.subplots(1, 3, figsize=(8, 3))
for ax, nlayers in zip(axs, [1, 2, 3]):
    sns.scatterplot(
        data=vis.add_jitter(data[data['ffnn.num_layers'] == nlayers], hyperpars, amount=0),
        x='val.macro avg.precision',
        y='val.macro avg.recall',
        hue='nn.learning_rate.log10',
        edgecolor='black',
    #     legend=False,
        s=10,
        palette='Spectral',
        ax=ax,
    )
    ax.set(
        title=f"{nlayers}-layer FFNNs",
        xlim=(-0.05, 1.05),
        ylim=(-0.05, 1.05),
    )
    vis.add_grid(ax)
    vis.fmt_legend(ax)
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=p,y=r,h=lr,c=nlayers.pdf'
)
plt.show()

#### x=lr y=f1,h=lr,c=nlayers

In [ ]:
# Three clusters in top right of precision-recall plots 
# -> caused by learning rate which needs to be _just_ right.
fig, axs = plt.subplots(1, 3, figsize=(8, 3))
for ax, nlayers in zip(axs, [1, 2, 3]):
    sns.scatterplot(
        data=vis.add_jitter(
            data[data['ffnn.num_layers'] == nlayers], 
            hyperpars, 
            amount=0
        ),
        x='nn.learning_rate.log10',
        y='val.macro avg.f1-score',
        hue='nn.learning_rate.log10',
        edgecolor='black',
#         legend=False,
        s=10,
        palette='Spectral',
        ax=ax,
    )
    ax.set(
        title=f"{nlayers}-layer FFNNs",
#         xlim=(-0.05, 1.05),
        ylim=(-0.05, 1.05),
        xticks=np.arange(-6, -1, .5)
    )
    vis.add_grid(ax)
    vis.fmt_legend(ax)
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=lr,y=f1,h=lr,c=nlayers.pdf'
)
plt.show()

#### x=lr,y=f1,h=npl-1,c=nlayers

In [ ]:
# The last layer of the FFNNs have a big impact on the overall performance
# Generally the best-performing FFNNs have many nodes in the last layer, 
# and reducing the number of nodes reduces the performance
fig, axs = plt.subplots(1, 3, figsize=(8, 3))
for ax, nlayers in zip(axs, [1, 2, 3]):
    sns.scatterplot(
        data=vis.add_jitter(data[data['ffnn.num_layers'] == nlayers], hyperpars, amount=0),
        x='nn.learning_rate.log10',
        y='val.macro avg.f1-score',
        hue='ffnn.nodes_per_layer.-1.log10',
        edgecolor='black',
#         legend=False,
        s=10,
        palette='Spectral',
        ax=ax,
    )
    ax.set(
        title=f"{nlayers}-layer FFNNs",
#         xlim=(-0.05, 1.05),
        ylim=(-0.05, 1.05),
        xticks=np.arange(-6, -1, 1)
    )
    vis.add_grid(ax)
    vis.fmt_legend(ax)
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=lr,y=f1,h=npl-1,c=nlayers.pdf'
)
plt.show()

#### x=lr,y=npl-1,h=f1,c=nlayers

In [ ]:
# nodes in last layer vs learning rate and F_1
fig, axs = plt.subplots(1, 3, figsize=(8, 3), dpi=300)
for ax, nlayers in zip(axs, [1, 2, 3]):
    sns.scatterplot(
        data=vis.add_jitter(data[data['ffnn.num_layers'] == nlayers], hyperpars, amount=0.05),
        x='nn.learning_rate.log10',
        hue='val.macro avg.f1-score',
        y='ffnn.nodes_per_layer.-1.log10',
        edgecolor='black',
        legend=nlayers == 1,
        s=10,
        palette='Spectral',
        ax=ax,
    )
    if nlayers == 1:
        vis.fmt_legend(ax, title='$F_1$')
        
    ax.set(
        title=f"{nlayers}-layer FFNNs\n(last layer)",
#         xlim=(-0.05, 1.05),
#         ylim=(-0.05, 1.05),
#         xticks=np.arange(-6, -1, 1)
    )
    vis.add_grid(ax)
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=lr,y=npl-1,h=f1,c=nlayers.pdf'
)
plt.show()

#### x=npl1,y=npl2,h=f1,c=nlayers

In [ ]:
# More nodes generally means they do better
fig, axs = plt.subplots(2, 2, figsize=(6, 6))

sns.stripplot(
    data=vis.add_jitter(
        data[data['ffnn.num_layers'] == 1], 
        hyperpars, 
    ),
    x='ffnn.nodes_per_layer.1.log10',
    hue='val.macro avg.f1-score',
    s=3,
    edgecolor='black',
    linewidth=.25,
#     legend=False,
    jitter=.125,
    palette='Spectral',
    ax=axs[0, 0]
)
vis.fmt_legend(axs[0, 0], title='$F_1$', loc='upper left')

sns.scatterplot(
    data=vis.add_jitter(
        data[data['ffnn.num_layers'] == 2], 
        hyperpars, 
    ),
    x='ffnn.nodes_per_layer.1.log10',
    y='ffnn.nodes_per_layer.2.log10',
    hue='val.macro avg.f1-score',
    s=10,
    edgecolor='black',
    legend=False,
    palette='Spectral',
    ax=axs[0, 1]
)

sns.scatterplot(
    data=vis.add_jitter(
        data[data['ffnn.num_layers'] == 3], 
        hyperpars, 
    ),
    x='ffnn.nodes_per_layer.1.log10',
    y='ffnn.nodes_per_layer.2.log10',
    hue='val.macro avg.f1-score',
    s=10,
    edgecolor='black',
    legend=False,
    palette='Spectral',
    ax=axs[1, 0]
)

sns.scatterplot(
    data=vis.add_jitter(
        data[data['ffnn.num_layers'] == 3], 
        hyperpars, 
    ),
    y='ffnn.nodes_per_layer.2.log10',
    x='ffnn.nodes_per_layer.3.log10',
    hue='val.macro avg.f1-score',
    s=10,
    edgecolor='black',
    legend=False,
    palette='Spectral',
    ax=axs[1, 1]
)

for ax, nlayers in zip(axs.flatten(), [1, 2, 3, 3]):
    ax.set(
        title=f'{nlayers}-layer FFNNs',
        xlabel=rename_hpars.get(ax.get_xlabel(), None),
        ylabel=rename_hpars.get(ax.get_ylabel(), None),
    )
    vis.add_grid(ax)

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=npl1,y=npl2,h=f1,c=nlayers.pdf'
)
plt.show()

#### x=npl1,y=npl2,z=npl3,h=f1

In [ ]:
# 3D plot of npl1 vs npl2 vs npl3 for 3-layer FFNNs
fig = plt.figure(dpi=200, figsize=(WIDTH*0.5, WIDTH*0.5))
ax = fig.add_subplot(projection='3d')

data_ = vis.add_jitter(
    data.loc[data['ffnn.num_layers'] == 3],
    ['ffnn.nodes_per_layer.1.log10', 'ffnn.nodes_per_layer.2.log10', 'ffnn.nodes_per_layer.3.log10'],
    amount=0.075
)

ax.scatter(
    data_['ffnn.nodes_per_layer.1.log10'],
    data_['ffnn.nodes_per_layer.2.log10'],
    data_['ffnn.nodes_per_layer.3.log10'],
    c=data_['val.macro avg.f1-score'],
    s=1,
    alpha=1,
    cmap='Spectral'
)
ax.set(
    xlabel='\#Nodes (layer 1, $\log_{10}$)',
    ylabel='\#Nodes (layer 2, $\log_{10}$)',
    zlabel='\#Nodes (layer 3, $\log_{10}$)',
    xlim=(0.4, 2.75),
    ylim=(0.4, 2.75),
    zlim=(0.4, 2.75),
)

ax.view_init(35, -45)

plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=npl1,y=npl2,z=npl3,h=f1.pdf',
    bbox_inches='tight',
)
# ax.set_box_aspect(None, zoom=0.9)
# fig.colorbar(C, ax=ax, fraction=0.02, pad=0.1, label='Name [units]')

#### x=dropout,y=npl-1,h=npl1,c=nlayers

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(2*WIDTH,2*WIDTH*.3))
for ax, nlayers in zip(axs, [1,2,3]):
    sns.scatterplot(
        data=vis.add_jitter(data[data['ffnn.num_layers'] == nlayers], hyperpars, amount=0.05),
        x='ffnn.dropout_rate',
        hue='val.macro avg.f1-score',
        y='ffnn.nodes_per_layer.-1.log10',
        s=20,
        edgecolor='black',
    #     legend=False,
        palette='Spectral',
        ax=ax,
    )
    vis.fmt_legend(ax)
    vis.add_grid(ax)
    ax.set(
        title=f'{nlayers}-layer FFNNs\n'
              f'(coloured by {rename_hpars["val.macro avg.f1-score"]})',
    )
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51fffnn,x=dropout,y=npl-1,h=npl1,c=nlayers.pdf',
    bbox_inches='tight',
)
plt.show()

In [ ]:
?cb.outline

In [ ]:
# Not for publication -- too big.
# x=dropout
# y=nodes per layer
# cols = nlayers
# rows = layer index

fig, axs = plt.subplots(3, 3, figsize=(10,10))

for i, nlayers in enumerate([1,2,3]):
    for j, layer_idx in enumerate([1,2,3]):
        ax = axs[i, j]
        if layer_idx > nlayers:
            ax.axis('off')
            continue

        data_ = data[data['ffnn.num_layers'] == nlayers]
        
        sns.scatterplot(
            data=vis.add_jitter(data_, hyperpars, amount=0.05),
            x='ffnn.dropout_rate',
            hue='val.macro avg.f1-score',
            y=f'ffnn.nodes_per_layer.{layer_idx}.log10',
            s=20,
            legend=False,
            edgecolor='black',
            palette='Spectral',
            ax=ax,
            hue_norm=(0, 1),
#             vmin=0,
#             vmax=1,
        )
        
        ax.set(
            title=f'{nlayers}-layer FFNNs, layer {layer_idx}',
            xlabel=None if nlayers != 3 else ax.get_xlabel(),
#             xticks=np.round(ax.get_xticks(), 2),
#             xticklabels=[] if nlayers != 3 else np.round(ax.get_xticks(), 2),
            ylabel=None if layer_idx != 1 else ax.get_ylabel(),
#             yticks=np.round(ax.get_yticks(), 2),
#             yticklabels=[] if layer_idx != 1 else np.round(ax.get_yticks(), 2),
        )
#         vis.fmt_legend(ax)
        vis.add_grid(ax)
        vis.add_cbar(
            fig, ax, data['val.macro avg.f1-score'], 
            label='$F_1$', vmin=0, vmax=1,
        )
plt.tight_layout()

#### x=dropout,y=f1,h=l2,c=nlayers

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(2*WIDTH,2*WIDTH*.3))
for ax, nlayers in zip(axs, [1,2,3]):
    sns.scatterplot(
        data=vis.add_jitter(data[data['ffnn.num_layers'] == nlayers], hyperpars, amount=0.05),
        x='ffnn.dropout_rate',
        y='val.macro avg.f1-score',
        hue='ffnn.l2_coefficient.log10',
        edgecolor='black',
        s=10,
        palette='Spectral',
        ax=ax,
#         legend=False
    )
    vis.fmt_legend(ax)
    vis.add_grid(ax)
    ax.set(
        title=f'{nlayers}-layer FFNNs\n(coloured by {rename_hpars["ffnn.l2_coefficient.log10"]})',
        ylabel=None if nlayers != 1 else ax.get_ylabel(),
        yticks=ax.get_yticks(),
        yticklabels=[] if nlayers != 1 else np.round(ax.get_yticks(), 1),
    )

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=dropout,y=f1,h=l2,c=nlayers.pdf',
    bbox_inches='tight',
)
plt.show()

#### loss ratios

In [ ]:

fig, axs = plt.subplots(1, 2, figsize=(8, 4))
# for ax, nlayers in zip(axs, [1, 2, 3]):
sns.scatterplot(
    data=data,
#     data=data[data['ffnn.num_layers'] == nlayers],
    x=np.log10(data['trn.loss']),
    y=np.log10(data['val.loss']),
    edgecolor='black',
    hue='ffnn.num_layers',
#         style='ffnn.num_layers',
    palette='tab10',
    s=10,
    ax=axs[0],
)
axs[0].plot(
    [-1.25, 0.5],
    [-1.25, 0.5],
    c='black',
    alpha=0.5,
    linewidth=1,
)
vis.add_grid(axs[0])
vis.fmt_legend(axs[0], title='\#Layers')
axs[0].set(
    xlabel='Training Loss ($\\log_{10}$)',
    ylabel='Validation Loss ($\\log_{10}$)',
)

sns.scatterplot(
    data=data,
#     data=data[data['ffnn.num_layers'] == nlayers],
    y=(data['ratio.loss']),
    x=np.log10(data['trn.loss']),
    edgecolor='black',
    hue='ffnn.num_layers',
#         style='ffnn.num_layers',
    palette='tab10',
    s=10,
    ax=axs[1],
)
vis.add_grid(axs[1])
vis.fmt_legend(axs[1], title='\#Layers')
axs[1].plot(
    [-1.25, 1.0],
    [1, 1],
    c='black',
    alpha=0.5,
    linewidth=1,
)
axs[1].set(
    ylabel='$\\frac{Training}{Validation}$ Loss',
    xlabel='Training Loss ($\\log_{10}$)',
)
plt.suptitle(
    f'Training and Validation loss'
)

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=trnloss,y=valloss,h=nlayers.pdf',
    bbox_inches='tight',
)

# ratio = training / validation
# low ratio -> training lower than validation -> overfitting (maybe)

#### 1-layer FFNNs

In [ ]:
data1 = data[data['ffnn.num_layers'] == 1]

In [ ]:
# Precision-recall plots for 1-layer FFNNs with hue for each hyperparameter
fig, axs = plt.subplots(2, 4, figsize=(8,4))
axs = axs.flatten()
for hpar, ax in zip(hyperpars, axs):
    sns.scatterplot(
        data=vis.add_jitter(data1, hyperpars, amount=0.1),
        x='val.macro avg.precision',
        y='val.macro avg.recall',
        hue=hpar,
        s=5,
        edgecolor=None,
        legend=False,
        palette='Spectral',
        ax=ax,
    )
    ax.set_title(rename_hpars[hpar])
    vis.add_grid(ax)
plt.tight_layout()
plt.show()

#### 2-layer FFNNs

In [ ]:
data2 = data[data['ffnn.num_layers'] == 2]

In [ ]:
# Precision-recall plots for 2-layer FFNNs with hue for each hyperparameter
fig, axs = plt.subplots(3, 3, figsize=(8,8))
axs = axs.flatten()
for hpar, ax in zip(hyperpars, axs):
    sns.scatterplot(
        data=vis.add_jitter(data2, hyperpars, amount=0.1),
        x='val.macro avg.precision',
        y='val.macro avg.recall',
        hue=hpar,
        s=10,
        edgecolor='black',
#         legend=False,
        palette='Spectral',
        ax=ax,
    )
    vis.fmt_legend(ax)
    ax.set_title(rename_hpars[hpar])
    vis.add_grid(ax)
plt.tight_layout()
plt.show()

#### 3-layer FFNNs

In [ ]:
data3 = data[data['ffnn.num_layers'] == 3]

In [ ]:
# Precision-recall plots for 3-layer FFNNs with hue for each hyperparameter
fig, axs = plt.subplots(2, 4, figsize=(8,4))
axs = axs.flatten()
for hpar, ax in zip(hyperpars, axs):
    sns.scatterplot(
        data=vis.add_jitter(data3, hyperpars, amount=0.1),
        x='val.macro avg.precision',
        y='val.macro avg.recall',
        hue=hpar,
        s=5,
        edgecolor=None,
        legend=False,
        palette='Spectral',
        ax=ax,
    )
    ax.set_title(rename_hpars[hpar])
    vis.add_grid(ax)
plt.tight_layout()
plt.show()

## Old in-depth FFNN plots

In [ ]:
(
    df.loc[(df['model_type'] == 'FFNN')]
    .groupby(['preprocessing.num_gesture_classes'])
    [['val.macro avg.f1-score', 'val.macro avg.precision', 'val.macro avg.recall']]
    .agg(['count', 'median', 'mean', 'min', 'max', 'std'])
    .T
    .round(3)
)

In [ ]:
data = df[
    (df['preprocessing.num_gesture_classes'] == '51') 
    & (df['model_type'] == 'FFNN')
]
data['Recall $>$ 0.7'] = data['val.macro avg.recall'] > 0.7

hyperpars = [
    'nn.learning_rate.log10',
    'nn.batch_size.log10',
    'ffnn.num_layers',
    'ffnn.nodes_per_layer.1.log10',
    'ffnn.nodes_per_layer.1',
    'ffnn.l2_coefficient.log10',
    'ffnn.dropout_rate',
    'ffnn.nodes_per_layer.2.log10',
    'ffnn.nodes_per_layer.3.log10',
]

In [ ]:
data = df[
    (df['preprocessing.num_gesture_classes'] == '51') 
    & (df['model_type'] == 'FFNN')
]

data['ffnn.nodes_per_layer.2'] = data['ffnn.nodes_per_layer.2'].fillna(0)
data['ffnn.nodes_per_layer.2.log10'] = data['ffnn.nodes_per_layer.2.log10'].fillna(0)
data['ffnn.nodes_per_layer.3'] = data['ffnn.nodes_per_layer.3'].fillna(0)
data['ffnn.nodes_per_layer.3.log10'] = data['ffnn.nodes_per_layer.3.log10'].fillna(0)
# data['recall gt 0.7'] = (data['val.macro avg.recall'] >= 0.7)

true = (data['val.macro avg.recall'] >= 0.7)
# Not easily seperable
# data['ffnn.nodes_per_layer.1.log10'].between(1.079181, 1.230449)

In [ ]:
# Split base on the manually found clusters
# clusters = [
#     [
#         {
#             'var': 'ffnn.nodes_per_layer.1',
#             'between': (12, 17),
#         }, {
#             'var': 'nn.learning_rate.log10',
#             'between': (-3.8, -2.95),
#         }, {
#             'var': 'ffnn.dropout_rate',
#             'between': (None, 0.4),
#         }
#     ], [
#         {
#             'var': 'ffnn.nodes_per_layer.1',
#             'between': (30, 100),
#         }, {
#             'var': 'ffnn.num_layers',
#             'between': (0.95, 1.05),
#         }, {
#             'var': 'nn.learning_rate.log10',
#             'between': (-5, -3),
#         }
#     ], [
#         {
#             'var': 'ffnn.nodes_per_layer.1',
#             'between': (160, 220),
#         }, {
#             'var': 'nn.learning_rate.log10',
#             'between': (-5.25, -2.5),
#         }, {
#             'var': 'ffnn.nodes_per_layer.3',
#             'between': (10, None),
#         }, {
#             'var': 'ffnn.nodes_per_layer.3',
#             'between': (None, 3),
#         },
#     ], [
#         {
#             'var': 'ffnn.nodes_per_layer.1',
#             'between': (280, None),
#         }, {
#             'var': 'ffnn.num_layers',
#             'between': (0.95, 2.05),
#         }, {
#             'var': 'nn.batch_size.log10',
#             'between': (None, 2.25),
#         }
#     ]
# ]
predictions = 'alt-manual'

if predictions == 'manual':

    mask_1st_layer_1_cluster = (
          (data['ffnn.nodes_per_layer.1'].between(12, 17))
        & (data['nn.learning_rate.log10'].between(-3.8, -2.95))
        & (data['ffnn.dropout_rate'] < 0.4)
    )
    mask_2nd_layer_1_cluster = (
          (data['ffnn.nodes_per_layer.1'].between(30, 100))
        & (data['nn.learning_rate.log10'].between(-5, -3))
        & (data['ffnn.num_layers'] == 1)
    )
    mask_3rd_layer_1_cluster = (
          (data['ffnn.nodes_per_layer.1'].between(160, 220))   
        & (data['nn.learning_rate.log10'].between(-5.25, -2.5))
        & (~data['ffnn.nodes_per_layer.3'].between(3, 10))
    )
    mask_4th_layer_1_cluster = (
          (data['ffnn.nodes_per_layer.1'] > 280)
        & (data['nn.batch_size.log10'] < 2.25)
        & (data['ffnn.num_layers'] != 3)
    )
    mask_layer_1_clusters = (
        (np.full(true.shape, False))
        | mask_1st_layer_1_cluster
        | mask_2nd_layer_1_cluster
        | mask_3rd_layer_1_cluster
        | mask_4th_layer_1_cluster
    )
    pred = (
        mask_layer_1_clusters
    )
elif predictions == 'alt-manual':
    pred = (
        (np.full(true.shape, False))
#         | (data['nn.learning_rate.log10'].between(-4.25, -2.5))
#         & (data['ffnn.num_layers'] == 1)
#         & (data['nn.learning_rate.log10'].between(-4, -2))

    )
elif predictions == 'svm':
    # Make predictions with a SVM classifier
    from sklearn.pipeline import make_pipeline
    from sklearn.preprocessing import StandardScaler
    clf = make_pipeline(StandardScaler(), svm.LinearSVC())
    clf = clf.fit(
        data[hyperpars].fillna(0),
        (data['val.macro avg.recall'] >= 0.7),
    )
    pred = clf.predict(data[hyperpars])
elif predictions == 'tree':
    from sklearn import tree

    dtree = tree.DecisionTreeClassifier(
        criterion='gini',
#         min_samples_split=2,
#         min_samples_leaf=None,
        max_leaf_nodes=5,
        max_depth=4,
        random_state=42,
    )
    clf = dtree.fit(
        data[hyperpars], 
        (data['val.macro avg.recall'] >= 0.7)
    )
    pred = clf.predict(data[hyperpars])

hue = np.empty_like(true, dtype=object)
hue[ true &  pred] = 'TP'
hue[~true & ~pred] = 'TN'
hue[~true &  pred] = 'FP'
hue[ true & ~pred] = 'FN'

data['conf_quad'] = hue
molten_data = data.melt(
    id_vars=['conf_quad'], 
    value_vars=hyperpars,
)

# Plot each hyperparameter and it's TN/FN/TP/FP options
g = sns.catplot(
    data=molten_data, 
    x="value", 
    row="variable", 
    hue="conf_quad",
    hue_order=['FN', 'FP', 'TN', 'TP'],
    palette=['#d81d04', '#d86b04', '#79d804', '#04d895'],
    kind="strip",
    sharex=False,
    dodge=True,
    size=2.5,
    alpha=0.5,
    jitter=0.25,
    height=1.5,
    aspect=4,
)
for ax in g.axes.flatten():
    xticks = ax.get_xticks()
    ax.set_xticks(np.round(np.linspace(
        xticks[0],
        xticks[-1],
        15,
    ), 2))

print(sklearn.metrics.confusion_matrix(true, pred))
if (hue == 'FP').sum() == 0: print("SUCCESS")
plt.show()

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)
sns.scatterplot(
    data=data,
    x='val.macro avg.precision',
    y='val.macro avg.recall',
    edgecolor=None,
    hue="conf_quad",
    hue_order=['FN', 'FP', 'TN', 'TP'],
    palette=['#d81d04', '#d86b04', '#79d804', '#04d895'],
    s=5,
#     alpha=0.5,
    ax=ax,
)
ax.set_yticks(np.arange(0, 1, 0.1))
ax.set_xticks(np.arange(0, 1, 0.1))
ax.grid(axis='both')
ax.set(
    xlim=(-0.05, 1.05),
    ylim=(-0.05, 1.05),
    xlabel="Precision",
    ylabel="Recall",
)

f1 = sklearn.metrics.f1_score(true, pred)
ax.set_title(f'{predictions.title()} Classifier f1={f1:.4f}')

### Decision tree classifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import tree

dtree = tree.DecisionTreeClassifier(
    criterion='gini',
#     max_depth=3,
)
distributions = dict(
    min_samples_split=[2, 4, 8, 16, 32, 64, 128],
    min_samples_leaf=[1, 2, 4, 8, 16, 32, 64, 128],
    max_leaf_nodes=[2, 3, 4, 5], # 10, 20, 40],
    max_depth=[2, 3, 4, 5,], # 8, 16, 32, 64],
)
clf = GridSearchCV(
    dtree, 
    distributions, 
#     random_state=42, 
    scoring='f1',
    verbose=1,
    error_score='raise',
#     n_iter=10000,
)

search = clf.fit(
    data[hyperpars], 
    (data['val.macro avg.recall'] >= 0.7)
)
search.best_score_, search.best_params_

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
tree.plot_tree(clf.best_estimator_, feature_names=hyperpars)
plt.show()

### Pairplot with recall highlighted

In [ ]:
hyperpars = [
    'ffnn.dropout_rate',
    'ffnn.l2_coefficient.log10',
    'nn.batch_size.log10',
    'nn.learning_rate.log10',
    'ffnn.num_layers',
    'ffnn.nodes_per_layer.1.log10',
    'ffnn.nodes_per_layer.2.log10',
    'ffnn.nodes_per_layer.3.log10',
]

data=df[
    (df['preprocessing.num_gesture_classes'] == '51') 
    & (df['model_type'] == 'FFNN')
]
data['recall $>=$ 0.7'] = (data['val.macro avg.recall'] >= 0.7)



g = sns.pairplot(
    data=data.rename(columns=rename_hpars),
    vars=[rename_hpars[hpar] for hpar in hyperpars],
#     data=data,
#     vars=hyperpars,
#     + ['val.macro avg.recall', 'val.macro avg.precision', 'val.macro avg.f1-score'],
    hue='recall $>=$ 0.7',
    height=3,
#     palette='Spectral',
#     diag_kind='hist',
    plot_kws={
        's': 10,
        'alpha': 0.5,
        'edgecolor': None,
    },
#     diag_kws={'hue': None, 'palette': None}
)

# for i, hpar_y in enumerate(hyperpars):
#     min_y = data[hpar_y].min()
#     max_y = data[hpar_y].max()
#     for j, hpar_x in enumerate(hyperpars):
#         if i == j: continue
# #         print(i, j, g.axes[i, j].get_xlabel(), hpar_y, hpar_x, g.axes[i, j].get_ylabel())
#         ax = g.axes[i, j]
#         min_x = data[hpar_x].min()
#         max_x = data[hpar_x].max()
#         for cluster in clusters:
#             for condition in cluster:
#                 if condition['var'] == hpar_x:
#                     lower = min_x if condition['between'][0] is None else condition['between'][0]
#                     upper = max_x if condition['between'][1] is None else condition['between'][1]
#                     ax.fill_betweenx(
#                         (min_y, max_y),
#                         lower,
#                         upper,
#                         color='tab:green',
#                         alpha=0.5,
#                     )
# #                     print('  performing condition for x', condition)
#                 if condition['var'] == hpar_y:
#                     lower = min_y if condition['between'][0] is None else condition['between'][0]
#                     upper = max_y if condition['between'][1] is None else condition['between'][1]
#                     ax.fill_between(
#                         (min_x, max_x),
#                         lower,
#                         upper,
#                         color='tab:green',
#                         alpha=0.5,
#                     )

### Conf mats of recall plots

In [ ]:
# Prepare the data
nclasses = '51'
data = df[
    (df['preprocessing.num_gesture_classes'] == nclasses) 
    & (df['model_type'] == 'FFNN')
#     & (data['val.macro avg.precision'] >= 0.5)
]
data['highlight'] = (
    (data['val.macro avg.recall'] >= 0.7)
)
conf_mats = {}
conf_mat_totals = {}
true_class_dists = {}
true_class_dists = {}

hpar = 'highlight'

assert len(data[hpar].unique()) < 10

for i, row in data.iterrows():
    try:
        y_true, y_pred = get_npz_data_from_model('../' + row['model_dir'])
    except:
        y_true, y_pred = get_npz_data_from_model('./' + row['model_dir'])
        
    cm = tf.math.confusion_matrix(
        y_true.flatten(), 
        y_pred.flatten()
    ).numpy()
    f1_score = sklearn.metrics.f1_score(
        y_true.flatten(), 
        y_pred.flatten(),
        average='macro',
        zero_division=0,
    )
    hpar_item = row[hpar]
    
    if hpar_item in conf_mats:
        conf_mats[hpar_item] += cm.astype(float) * f1_score
        conf_mat_totals[hpar_item] += f1_score
    else:
        conf_mats[hpar_item] = cm.astype(float) * f1_score
        conf_mat_totals[hpar_item] = f1_score

In [ ]:
# Make the plots
fig, axs = plt.subplots(
    1,2,
    figsize=(WIDTH, WIDTH*0.5)
)
print(conf_mats.keys())
axs = axs.flatten()
for i, (hpar_item, conf_mat) in enumerate(conf_mats.items()):
#     conf_mat[-1, -1] = 0
#     conf_mat /= conf_mat_totals[hpar_item]
#     conf_mat /= conf_mat.max()
#     print(conf_mat_totals[hpar_item])
    # Normalize the matrix manually to get nicer annotations
    vis.conf_mat(
        conf_mat
        / conf_mat.sum(axis=0), 
        ax=axs[i], 
        norm=None
    )
#     axs[i].set_title(
#         f'{hpar_item} Confusion Matrices\n(weighted mean, {nclasses} classes)'
#     )
# axs[0].set_title(r'Recall $>=$ 0.7')
# axs[1].set_title(r'Recall $<$ 0.7')

plt.tight_layout()
plt.suptitle("Precision $>$ 0.5")
axs[0].set_title("Recall $>$ 0.7")
axs[1].set_title("Recall $<$ 0.7)")
plt.show()

### X-class FFNN learning rate vs f1

In [ ]:
nclasses_list = ('5','50','51')
for nclasses in nclasses_list:
    data = df[
        (df['preprocessing.num_gesture_classes'] == nclasses) 
        & (df['model_type'] == 'FFNN')
    ]

    fig, axs = plt.subplots(1, 2, figsize=(WIDTH, WIDTH*0.5))
    sns.scatterplot(
        data=data,
        y='val.macro avg.f1-score',
        x='nn.learning_rate.log10',
        hue='val.loss.log10',
        palette='Spectral_r',
        s=5,
        edgecolor=None,
        ax=axs[0],
    #     alpha=0.5,
    )
    print(axs[0].get_xlim())
    axs[0].set(
        title=f'Learning Rate vs $F_1$-score\n{nclasses}-class FFNN',
        xlabel='Learning Rate ($\log_{10}$)',
        ylabel=rename_hpars['val.macro avg.f1-score'],
        ylim=(-0.05, 1.05),
        xlim=(axs[0].get_xlim()[0] - 3, axs[0].get_xlim()[1]),
    )
    print(axs[0].get_xlim())
    axs[0].legend().set_title('Loss ($\log_{10}$)')

    sns.scatterplot(
        data=data,
        hue='val.macro avg.f1-score',
        x='nn.learning_rate.log10',
        y='val.loss.log10',
        palette='Spectral',
        s=5,
        edgecolor=None,
        ax=axs[1],
    #     alpha=0.5,
    )
    axs[1].legend().set_title('$F_1$')
    axs[1].set(
        title=f'Learning Rate vs Validation Loss\n{nclasses}-class FFNN',
        xlabel='Learning Rate ($\log_{10}$)',
        ylabel=rename_hpars['val.loss.log10'],
        xlim=(axs[1].get_xlim()[0] - 3, axs[1].get_xlim()[1]),
    )
    plt.tight_layout()
    plt.savefig(
        f'../../report/src/imgs/graphs/05_hpar_analysis_ffnn_classes{nclasses}_lr.pdf',
        bbox_inches='tight',
    )
    plt.show()


In [ ]:
# Find the best learning rate for 51-class FFNNs to get the lowest validation loss
(
    df[
        (df['preprocessing.num_gesture_classes'] == '51') 
        & (df['model_type'] == 'FFNN')
    ].sort_values('val.loss')
    [['nn.learning_rate.log10', 'val.loss.log10', 'val.macro avg.f1-score']]
).head()

#### 50&51-class FFNN Num layers vs f1

In [ ]:

nclasses_list = ('50','51')
for nclasses in nclasses_list:
    data = df[
        (df['preprocessing.num_gesture_classes'] == nclasses) 
        & (df['model_type'] == 'FFNN')
    ]

    fig, axs = plt.subplots(1, 2, figsize=(WIDTH, WIDTH*0.5))
    sns.stripplot(
        data=data,
        y='val.macro avg.f1-score',
        x='ffnn.num_layers',
        hue='val.loss.log10',
        palette='Spectral_r',
        jitter=0.25,
        s=3,
        ax=axs[0],
#         alpha=0.5,
    )
    axs[0].set(
        title=f'\#Layers vs $F_1$-score\n{nclasses}-class FFNN',
        xlabel='Number of Layers',
        ylabel=rename_hpars['val.macro avg.f1-score'],
        ylim=(-0.05, 1.05),
        xlim=(-2.5, 2.5),
    )
    handles, labels = axs[0].get_legend_handles_labels()
    axs[0].legend(
        handles=handles[0::2], 
        labels=labels[0::2],
        loc='center left',
        title='Loss ($\log_{10}$)',
    )

    sns.stripplot(
        data=data,
        hue='val.macro avg.f1-score',
        x='ffnn.num_layers',
        y='val.loss.log10',
        palette='Spectral',
        jitter=0.25,
        s=3,
        ax=axs[1],
#         alpha=0.5,
    )

    handles, labels = axs[1].get_legend_handles_labels()
    axs[1].legend(
        handles=handles[0::2], 
        labels=labels[0::2],
        title='$F_1$',
        loc='center left'
    )

    
    axs[1].set(
        title=f'\#Layers vs Validation Loss\n{nclasses}-class FFNN',
        xlabel='Number of Layers',
        ylabel=rename_hpars['val.loss.log10'],
        xlim=(-2, 2.5),
    )
    plt.tight_layout()
    plt.savefig(
        f'../../report/src/imgs/graphs/05_hpar_analysis_ffnn_classes{nclasses}_nlayers.pdf',
        bbox_inches='tight',
    )
    plt.show()


#### 51-class Num nodes

In [ ]:
def calc_num_weights(list_of_nodes, nclasses=51):
    l = list_of_nodes[:]
#     if len(list_of_nodes) == 1: return list_of_nodes[0]
    l.insert(0, 20*30)
    l.append(nclasses)
    result = 0
    for i in range(len(l)-1):
        result += l[i] * l[i+1]
    return result

fig, axs = plt.subplots(2, 2, figsize=(WIDTH, WIDTH))
xmin = np.inf
xmax = -np.inf
nclasses = '51'

hue_col = df.loc[
    (df['preprocessing.num_gesture_classes'] == nclasses) 
    & (df['ffnn.num_layers'].isin([1, 2, 3]))
    & (df['model_type'] == 'FFNN'),
    'val.loss.log10',
]

for layer_num in (1, 2, 3):
    data = df[
        (df['preprocessing.num_gesture_classes'] == nclasses) 
        & (df['model_type'] == 'FFNN')
        & (df['ffnn.num_layers'] == layer_num)
    ]

    data['num_weights'] = data['ffnn.nodes_per_layer'].apply(calc_num_weights)
#     data['weight_prod'] = data['ffnn.nodes_per_layer'].apply(
#         lambda x: 600 * np.prod(x) * 51
#     )
#     data['weight_sum'] =  data['ffnn.nodes_per_layer'].apply(
#         lambda x: 600 + np.sum(x) + 51
#     )
#     data['weight_prod.log10'] = data['weight_prod'].apply(np.log10)
#     data['weight_sum.log10'] = data['weight_sum'].apply(np.log10)

    custom_norm = mpl.colors.Normalize(
        vmin=-1.5,
        vmax=-0.4,
    )
    sns.scatterplot(
        data=data,
        x='num_weights',
        y='val.macro avg.f1-score',
        hue='val.loss.log10',
        hue_norm=custom_norm,
        palette='Spectral_r',
        s=5,
        edgecolor=None,
        ax=axs.flat[layer_num-1],
        legend=layer_num == 1,
#         legend=False,
    )
    xmin = min(xmin, axs.flat[layer_num-1].get_xlim()[0])
    xmax = max(xmax, axs.flat[layer_num-1].get_xlim()[1])
    
#     display(
#         data.loc[
#             data['val.macro avg.f1-score'] > 0.7,
#             ['num_weights', 'ffnn.nodes_per_layer']
#         ].sort_values('num_weights').head(20)
#     )

for i, ax in enumerate(axs.flat[:-1]):
    layer_s = 'layer' if i+1 == 1 else 'layers'
    ax.set(
        ylim=(-0.05, 1.05),
#         xlim=(ax.get_xlim()[0], ax.get_xlim()[1]*1.5),
        xlabel='\#Weights',
        ylabel='$F_1$-score',
        title=f'Number of Weights vs $F_1$-score\n51-class FFNNs, {i+1} {layer_s}',
    )

plt.tight_layout()
axs.flat[-1].axis('off')
move_legend(axs.flat[0], axs.flat[-1], title='Val. Loss ($\log_{10}$)')

plt.savefig(
    '../../report/src/imgs/graphs/05_hpar_analysis_ffnn_classes51_nweights.pdf',
    bbox_inches='tight',
)

plt.show()

In [ ]:
model_type = 'FFNN'
metrics = (
    'f1-score',
    'precision',
    'recall',
)

hyperpars = (
    'ffnn.dropout_rate',
    'ffnn.l2_coefficient.log10',
    'nn.batch_size.log10',
    'nn.learning_rate.log10',
    'ffnn.num_layers',
    'ffnn.nodes_per_layer.1',
    'ffnn.nodes_per_layer.2',
    'ffnn.nodes_per_layer.3',
)
val_loss_maxs = (
    .5,
    10,
    -0.25,
)

nclasses_list = ('5', '50', '51')
for nclasses, val_loss_max in zip(nclasses_list, val_loss_maxs):
    print(f"{nclasses} classes")
    data = df[
        (df['preprocessing.num_gesture_classes'] == nclasses) 
        & (df['model_type'] == model_type)
        & (df['val.loss.log10'] <= val_loss_max)
#         & (df['val.macro avg.f1-score'] >= 0.9)
    ]
    for metric in metrics:
        print(f"  {metric}")
        
        nrows = 3
        ncols = 3
        fig, axs = plt.subplots(
            nrows, ncols,
            figsize=(WIDTH, WIDTH),
        )
        handles = None
        labels = None
        for i, ax in enumerate(axs.flatten()):
            if len(hyperpars) <= i:
                if handles is not None and len(hyperpars) == i:
                    ax.legend(
                        handles=handles, 
                        labels=labels,
                        title=rename_hpars[common_kwargs['hue']],
                        loc='center',
                    )
                ax.axis('off')
                continue

            common_kwargs = {
                'data': data.sort_values(f'val.macro avg.{metric}', ascending=True),
                'x': hyperpars[i],
                'y': f'val.macro avg.{metric}',
                'hue': 'val.loss.log10',
                'palette': 'Spectral',
                'ax': ax,
            }
            
            if data[common_kwargs['x']].nunique() > 5:
                sns.scatterplot(
                    edgecolor=None,
                    s=5,
                    **common_kwargs,
                )
            else:
                sns.stripplot(
                    s=2.5,
                    jitter=.25,
                    **common_kwargs,
                )
            ax.set(
                title=(
                    f"{rename_hpars[common_kwargs['y']]} vs {rename_hpars[common_kwargs['x']]}"
                    .replace(" ($\\log_{10}$)", "")
                    .replace("Val. ", "")
                    .replace("-score", "")
                ),
                ylabel=rename_hpars[common_kwargs['y']],
                xlabel=rename_hpars[common_kwargs['x']],
            )
            if i % ncols != 0:
                ax.set(
                    ylabel=None,
                    yticks=[],
                )
            handles, labels = ax.get_legend_handles_labels()
            ax.legend().remove()
        plt.tight_layout()
        path = f'../../report/src/imgs/graphs/' + (
            f'05_hpar_analysis_{model_type.lower()}_classes{nclasses}_'
            f'y{common_kwargs["y"]}_hue{common_kwargs["hue"]}'
        ).replace(".", "_").replace("-", '_').replace(" ", '_') + '.pdf'
        print(f'    {path}')
        plt.savefig(path, bbox_inches='tight')
        plt.show()

### Pairplot of FFNN hyperparameters

In [ ]:
sns.pairplot(
    data=vis.add_jitter(data, hyperpars).rename(columns=rename_hpars),
    hue=rename_hpars['val.macro avg.f1-score'],
    vars=[rename_hpars[hpar] for hpar in list(hyperpars)] + [rename_hpars['val.macro avg.f1-score']],
    height=1,
    palette='Spectral',
    diag_kind='hist',
    plot_kws={
        's': 2.5,
        'alpha': 0.5,
        'edgecolor': None,
    },
    diag_kws={'hue': None, 'palette': None}
)

# plt.savefig(
#     '../../report/src/imgs/graphs/05_hpar_analysis_ffnn_pairplot.png',
#     bbox_inches='tight'
# )

### Hpars of interest for 51 classes FFNNs

In [ ]:
print("Hyperparameters of interest for 51 FFNN classes")
hpars = (
    'nn.learning_rate.log10',
    'ffnn.num_layers',
    'ffnn.nodes_per_layer.1',
    'ffnn.l2_coefficient.log10',
)
legend_titles = (
    'LR ($\log_{10}$)',
    '\#Layers',
    'Nodes (layer 1)',
    'L2 Coef ($\log_{10}$)',
)

fig, axs = plt.subplots(2, 2, figsize=(WIDTH, WIDTH))
axs = axs.flatten()

for i, (hpar, legend_title) in enumerate(zip(hpars, legend_titles)):
    sns.scatterplot(
        data=df[
            (df['preprocessing.num_gesture_classes'] == '51') 
            & (df['model_type'] == 'FFNN')
            & (df['val.loss.log10'] <= max_log10_val_loss)
        ],
        y='val.macro avg.recall',
        x='val.loss.log10',
#         hue=hpar,
        hue='ffnn.nodes_per_layer.1',
        palette='viridis',
        s=10,
        edgecolor=None,
        ax=axs[i]
    )
#     axs[i].set(
#         xlabel='Validation Loss ($\log_{10}$)',
#         ylabel='Recall',
#         title=f'{legend_title}\n(Recall vs Validation Loss)',
#         ylim=(-0.05, 1.05),
#     )
#     axs[i].legend().set_title(legend_title)

plt.tight_layout()

In [ ]:
hyperpars = (
    'ffnn.dropout_rate',
    'ffnn.l2_coefficient.log10',
    'nn.batch_size.log10',
    'nn.learning_rate.log10',
    'ffnn.nodes_per_layer.1',
    'ffnn.nodes_per_layer.2',
    'ffnn.nodes_per_layer.3',
)

fig, axs = plt.subplots(3, 3, figsize=(WIDTH, WIDTH))

for hpar, ax in zip(hyperpars, axs.flatten()):
    sns.scatterplot(
        data=df[
            (df['preprocessing.num_gesture_classes'] == '51') &
            (df['model_type'] == 'FFNN')
        ],
        x=hpar,
        y='val.loss.log10',
        hue='trn.loss.log10',
        ax=ax,
        palette='Spectral',
        s=10,
        alpha=0.5,
    edgecolor=None,
    )
    ax.legend().set_title('Training Loss\n($\log_{10}$)')
# plt.show()

sns.scatterplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51') &
        (df['model_type'] == 'FFNN')
    ],
    y='val.loss.log10',
    x='trn.loss.log10',
    ax=axs[-1, -1],
    edgecolor=None,
)
plt.tight_layout()

## In depth HFFNN plots



In [ ]:
maj_min_hyperpars = [
    'hffnn.{}.ffnn.dropout_rate',
    'hffnn.{}.ffnn.l2_coefficient.log10',
    'hffnn.{}.nn.batch_size.log10',
    'hffnn.{}.nn.learning_rate.log10',
    'hffnn.{}.ffnn.num_layers',
    'hffnn.{}.ffnn.nodes_per_layer.1.log10',
    'hffnn.{}.ffnn.nodes_per_layer.2.log10',
    'hffnn.{}.ffnn.nodes_per_layer.3.log10',
]

hyperpars = [
    hpar.format(subtype) 
    for subtype in ['majority', 'minority'] 
    for hpar in maj_min_hyperpars
]

data = df[
    (df['preprocessing.num_gesture_classes'] == '51') 
    & (df['model_type'] == 'HFFNN')
    & (df['hffnn.majority.ffnn.dropout_rate'].notna())
]
data['$F_1 >$ 0.5'] = data['val.macro avg.f1-score'] > 0.5

In [ ]:
aspect=2
g = sns.catplot(
    data=data.melt( 
        id_vars=['$F_1 >$ 0.5'], 
        value_vars=hyperpars,
    ), 
    x="value", 
    col="variable",
    col_wrap=2,
    hue="$F_1 >$ 0.5",
#     hue_order=[True, False],
#     palette=['#d81d04', '#d86b04', '#79d804', '#04d895'],
    kind="violin",
    sharex=False,
    inner='stick',
#     dodge=True,
#     size=2.5,
#     alpha=0.5,
#     jitter=0.25,
    split=True,
    height=WIDTH*0.5 / aspect,
    aspect=aspect,
)
for ax in g.axes.flat:
    hpar = ax.get_title().split(' = ')[1]
    ax.set_title(f'{rename_hpars[hpar]}')
plt.tight_layout()

# sns.move_legend(g, (0.7, 0.09))

# plt.savefig(
#     '../../report/src/imgs/graphs/05_hpar_analysis_hffnn_p_lt_0_05.pdf',
#     bbox_inches='tight',
# )

### Distribution plots

In [ ]:
mask = (
    np.full(len(data), True)
#     data['hffnn.minority.nn.learning_rate.log10'].between(-4, -1.5)
#     & data['hffnn.majority.nn.learning_rate.log10'].between(-4.5, -1.85)
#     & data['hffnn.minority.ffnn.nodes_per_layer.1.log10'].between(.85, 10)
)

of_interest = [
    'hffnn.majority.nn.learning_rate.log10',
    'hffnn.minority.nn.learning_rate.log10',
    'hffnn.majority.ffnn.num_layers',
    
    
]
fig, axs = plt.subplots(5, 2, figsize=(WIDTH, WIDTH))

for i, ax in enumerate(axs.flat):
    if i >= len(of_interest):
        ax.axis('off')
        continue
    sns.scatterplot(
        data=data[mask],
        y='val.macro avg.f1-score',
        x=of_interest[i],
        hue='$F_1 >$ 0.5',
        s=5,
        edgecolor=None,
        alpha=0.75,
        ax=ax,
#         legend=i == 0,
        legend=False,
    )
    p = ks_test_df.loc[
        ks_test_df['hpar'] == of_interest[i],
        'p_value'
    ].values[0]
    ax.set(
        ylim=(-0.05, 1.05),
        ylabel='$F_1$-score',
        xlabel=rename_hpars[of_interest[i]],
        title=(
            rename_hpars[of_interest[i]].replace(r' ($\log_{10}$)', '').replace(r', $\log_{10}$', '') 
            + f' (p={p:.1e})'
        )
    )
#     if i % 2 != 0:
#         ax.set_ylabel(None)
#         ax.set_yticks([])
plt.tight_layout()

# sns.move_legend(axs.flat[0], (1.45, -6.5))

# plt.savefig(
#     '../../report/src/imgs/graphs/05_hpar_analysis_hffnn_hpars.pdf',
#     bbox_inches='tight',
# )

plt.show()
sns.jointplot(
    data=data[data['hffnn.minority.nn.learning_rate.log10'].notna()],
    x='val.macro avg.precision',
    y='val.macro avg.recall',
    hue=mask,
    s=5,
    edgecolor=None,
)

In [ ]:
maj_min = (
    'majority',
    'minority',
)
for submodel in maj_min:
    hyperpars = (
        f'hffnn.{submodel}.ffnn.dropout_rate',
        f'hffnn.{submodel}.ffnn.l2_coefficient.log10',
        f'hffnn.{submodel}.nn.batch_size.log10',
        f'hffnn.{submodel}.nn.learning_rate.log10',
        f'hffnn.{submodel}.ffnn.nodes_per_layer.1',
        f'hffnn.{submodel}.ffnn.nodes_per_layer.2',
        f'hffnn.{submodel}.ffnn.nodes_per_layer.3',
    )
    df[[
        f'hffnn.{submodel}.ffnn.l2_coefficient.log10',
        f'hffnn.{submodel}.nn.batch_size.log10',
        f'hffnn.{submodel}.nn.learning_rate.log10',
    ]] = np.log10(df[[
        f'hffnn.{submodel}.ffnn.l2_coefficient',
        f'hffnn.{submodel}.nn.batch_size',
        f'hffnn.{submodel}.nn.learning_rate',
    ]])
    xlabels = (
        f'Dropout Rate\n({submodel.title()} Classifier)',
        f'L2 Coefficient ($\log_{{10}}$)\n({submodel.title()} Classifier)',
        f'Batch Size ($\log_{{10}}$)\n({submodel.title()} Classifier)',
        f'Learning Rate ($\log_{{10}}$)\n({submodel.title()} Classifier)',
        f'Nodes in Layer 1\n({submodel.title()} Classifier)',
        f'Nodes in Layer 2\n({submodel.title()} Classifier)',
        f'Nodes in Layer 3\n({submodel.title()} Classifier)',
    )

    data = df[
        (df['preprocessing.num_gesture_classes'] == '51') &
        (df['model_type'] == 'HFFNN')
    ]

    fig, axs = plt.subplots(
        3, 3,
        figsize=(WIDTH, WIDTH)
    )
    print(axs.shape)
    for i, ax in enumerate(axs.flatten()):
        if len(hyperpars) <= i:
            ax.axis('off')
            continue

        sns.scatterplot(
            data=data,
            x=hyperpars[i],
            y='val.macro avg.f1-score',
            s=10,
            alpha=0.5,
            ax=ax,
            color='tab:orange',
            edgecolor=None,
        )
        title_xlabel = xlabels[i].replace(" ($\log_{10}$)", "")
        ax.set(
            title=f'$F_1$ score vs {title_xlabel}',
            xlabel=xlabels[i],
            ylabel='$F_1$ score',
            ylim=(-0.1, 1.1),
        )
    plt.tight_layout()
#     plt.savefig(
#         f'../../report/src/imgs/graphs/05_in_depth_hffnn_{submodel}_hpars.pdf',
#         bbox_inches='tight',
#     )
    plt.show()

In [ ]:
?sns.move_legend

## In depth CuSUM plots

In [ ]:
nclasses_list = ('5', '50', '51')
for nclasses in nclasses_list:
    data = df[
        (df['preprocessing.num_gesture_classes'] == nclasses)
        & (df['model_type'] == 'CuSUM')
    ]

    fig, axs = plt.subplots(1, 2, figsize=(WIDTH, WIDTH*.5))
    p_min = max(-0.05, data['val.macro avg.precision'].min() / 1.10)
    p_max = min( 1.05, data['val.macro avg.precision'].max() * 1.10)
    r_min = max(-0.05, data['val.macro avg.recall'].min()    / 1.10)
    r_max = min( 1.05, data['val.macro avg.recall'].max()    * 1.10)

    recall_grid, precision_grid = np.meshgrid(
        np.linspace(p_min, p_max, 100), 
        np.linspace(r_min, r_max, 100)
    )
    f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

    contours = axs[0].contour(
        recall_grid, 
        precision_grid,
        f1_score, 
        levels=np.linspace(
            f1_score.min(),
            f1_score.max(),
            10,
        ), 
        colors='black',
        alpha=0.25
    )
    axs[0].clabel(contours, inline=True, fontsize=8, fmt='%.2f')

    sns.scatterplot(
        data=data,
        y='val.macro avg.recall',
        x='val.macro avg.precision',
        hue='cusum.thresh',
        alpha=0.75,
        s=10,
        ax=axs[0],
        palette=palette,
        edgecolor=None
    )

    sns.stripplot(
        data=data,
        x='cusum.thresh',
        y='val.macro avg.f1-score',
        alpha=0.5,
        s=2.5,
        ax=axs[1],
        jitter=0.5,
        color=model_colours['CuSUM'],
        native_scale=True,
    )

    p_range = p_max - p_min
    r_range = r_max - r_min
    axs[0].set(
        title=f"Precision vs recall\n{nclasses}-class CuSUM models",
        xlabel='Precision',
        ylabel='Recall',
        xlim=(p_min - p_range*0.025, p_max + p_range*0.025),
        ylim=(r_min - r_range*0.025, r_max + r_range*0.025),
    )

    axs[0].legend().set_title('Threshold')

    axs[1].set(
        title=f'Threshold vs $F_1$ score\n{nclasses}-class CuSUM models',
        xlabel='CuSUM Threshold',
        ylabel='$F_1$ score',
    )
    plt.tight_layout()
    plt.savefig(
        f'../../report/src/imgs/graphs/05_hpar_analysis_cusum_classes{nclasses}.pdf',
        bbox_inches='tight',
    )
    plt.show()

## In depth SVM plots

- Correlation with the training time per observation
- Correlation with the precision/recall/f1
- clusters in recall-precision space

In [ ]:
# Get some summary stats
(
    df.loc[(df['model_type'] == 'SVM')]
    .groupby(['preprocessing.num_gesture_classes'], dropna=False)
    [['val.macro avg.f1-score', 'val.macro avg.precision', 'val.macro avg.recall']]
    .agg(['count', 'median', 'mean', 'min', 'max', 'std'])
    .T
    .round(3)
)

In [ ]:
n_classes = ('5', '50', '51')
for num_gesture_classes in n_classes:
    data = df[
        (df['preprocessing.num_gesture_classes'] == num_gesture_classes)
        & (df['model_type'] == 'SVM')
    ]

    data['svm.c.log10'] = np.log10(data['svm.c'])
    data['svm.class_weight'] = data['svm.class_weight'].fillna('unbalanced')

    data = data.rename(columns={
        'svm.c.log10': '$\log_{10}(C)$', 
        'svm.class_weight': 'Class Weight'
    })

    fig, axs = plt.subplots(2, 2, figsize=(WIDTH, WIDTH))
    axs = axs.flatten()
    p_min = data['val.macro avg.precision'].min() / 1.05
    p_max = data['val.macro avg.precision'].max() * 1.05
    r_min = data['val.macro avg.recall'].min()    / 1.05
    r_max = data['val.macro avg.recall'].max()    * 1.05

    recall_grid, precision_grid = np.meshgrid(
        np.linspace(p_min, p_max, 100), 
        np.linspace(r_min, r_max, 100)
    )
    f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

    contours = axs[0].contour(
        recall_grid, 
        precision_grid,
        f1_score, 
        levels=np.linspace(
            f1_score.min(),
            f1_score.max(),
            5
        ), 
        colors='black',
        alpha=0.25
    )
    axs[0].clabel(contours, inline=True, fontsize=8, fmt='%.2f')

    sns.scatterplot(
        data=data,
        y='val.macro avg.recall',
        x='val.macro avg.precision',
        hue='$\log_{10}(C)$',
        style='Class Weight',
        alpha=0.8,
        s=10,
        ax=axs[0],
        palette=palette,
        edgecolor=None,
    )

    axs[0].set(
        title=f"Precision vs Recall"
        f"\n({num_gesture_classes}-class SVM)",
        xlabel='Precision',
        ylabel='Recall',
    )

    sns.scatterplot(
        data=data,
        x='$\log_{10}(C)$',
        y='val.macro avg.f1-score',
        style='Class Weight',
        alpha=0.8,
        s=10,
        ax=axs[1],
        color='tab:purple',
        legend=False,
        edgecolor=None,
    )

    axs[1].set(
        title=f"$F_1$-score vs C\n"
        f"({num_gesture_classes}-class SVM)",
        xlabel='C ($\log_{10}$)',
        ylabel='$F_1$ score',
    )
    
    move_legend(axs[0], axs[-1])
    axs[-2].axis('off')
    plt.tight_layout()
    plt.savefig(
        f'../../report/src/imgs/graphs/05_in_depth_svm_classes{num_gesture_classes}.pdf',
        bbox_inches='tight',
    )
    plt.show()

In [ ]:
nclasses_list = ('5', '50', '51')
for nclasses in nclasses_list:
    data = df[
        (df['preprocessing.num_gesture_classes'] == nclasses)
        & (df['model_type'] == 'SVM')
    ]

    data['svm.class_weight'] = data['svm.class_weight'].fillna('unbalanced')

    conf_mats = {}
    conf_mat_totals = {}
    precisions = {}
    recalls = {}
    f1_scores = {}

    hpar = 'svm.class_weight'

    assert len(data[hpar].unique()) < 10

    for i, row in data.iterrows():
        y_true, y_pred = get_npz_data_from_model('../' + row['model_dir'])
        cm = tf.math.confusion_matrix(
            y_true.flatten(), 
            y_pred.flatten()
        ).numpy()
        hpar_item = row[hpar]

        precision, recall, f1_score, _support = sklearn.metrics.precision_recall_fscore_support(
            y_true.flatten(), 
            y_pred.flatten(),
            zero_division=0,
        )


        if hpar_item in conf_mats:
            conf_mats[hpar_item] += cm.astype(float)
            conf_mat_totals[hpar_item] += 1.0
            precisions[hpar_item] += precision
            recalls[hpar_item] += recall
            f1_scores[hpar_item] += f1_score

        else:
            conf_mats[hpar_item] = cm.astype(float)
            conf_mat_totals[hpar_item] = 1.0
            precisions[hpar_item] = precision
            recalls[hpar_item] = recall
            f1_scores[hpar_item] = f1_score


    fig, axs = plt.subplots(
        1, len(conf_mats), 
        figsize=(WIDTH, WIDTH/len(conf_mats))
    )


    for i, (hpar_item, conf_mat) in enumerate(conf_mats.items()):
    #     conf_mat[-1, -1] = 0
        conf_mat /= conf_mat_totals[hpar_item]
        vis.conf_mat(conf_mat, ax=axs[i])
        axs[i].set_title(
            f'{hpar_item.title()} SVMs'
        )

    plt.tight_layout()

    plt.savefig(
        f'../../report/src/imgs/graphs/05_in_depth_svm_conf_mats_unbalanced_classes{nclasses}.pdf',
        bbox_inches='tight',
    )
    plt.show()

    fig, axs = plt.subplots(
        1, 2,
        figsize=(WIDTH, WIDTH*.2),
    )
    axs = axs.flatten()
    for i, (hpar_item, conf_mat) in enumerate(conf_mats.items()):
        axs[i].set_aspect(3)
        vis.precision_recall_f1(
            precision=precisions[hpar_item] / conf_mat_totals[hpar_item],
            recall=recalls[hpar_item] / conf_mat_totals[hpar_item],
            f1=f1_scores[hpar_item] / conf_mat_totals[hpar_item],
            ax=axs[i],
        )
        axs[i].set_title(hpar_item.title() + " SVMs")
    plt.tight_layout()

    plt.savefig(
        f'../../report/src/imgs/graphs/05_in_depth_svm_prf1_plots_unbalanced_classes{nclasses}.pdf',
        bbox_inches='tight',
    )
    plt.show()


In [ ]:
[c for c in df.columns if 'time' in c]

In [ ]:
data = df[
    (df['preprocessing.num_gesture_classes'] == '51')
    & (df['model_type'] == 'SVM')
]

data['svm.c.log10'] = np.log10(data['svm.c'])
data['svm.class_weight'] = data['svm.class_weight'].fillna('unbalanced')

data = data.rename(columns={
    'svm.c.log10': '$\log_{10}(C)$', 
    'svm.class_weight': 'Class Weight'
})
fig, axs = plt.subplots(1, 2, figsize=(WIDTH, WIDTH*0.5))

sns.scatterplot(
    data=data,
    y='fit_time_per_obs',
    x='$\log_{10}(C)$',
    s=10,
    alpha=0.5,
#     hue='',
    style='Class Weight',
    color='tab:purple',
    palette=palette,
    ax=axs[0],
    edgecolor=None,
)
sns.scatterplot(
    data=data,
    y='trn.pred_time_per_obs',
    x='$\log_{10}(C)$',
    s=10,
    alpha=0.5,
#     hue='',
    style='Class Weight',
    color='tab:purple',
    palette=palette,
    ax=axs[1],
    edgecolor=None,
    legend=False
)

axs[0].set(
    title='Fit time vs C\n51-class SVM',
    ylabel='Fit time (s/obs.)',
    xlabel='$\log_{10}(C)$',
)
axs[1].set(
    title='Inference time vs C\n51-class SVM',
    ylabel='Inference time (s/obs.)',
    xlabel='$\log_{10}(C)$',
)
plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_svm_hpars_vs_fit_time.pdf',
    bbox_inches='tight'
)

## In depth HMM plots

- Clusters in recall-precision space
- 

In [ ]:
# Get some summary stats
hue_order = ['Spherical', 'Diagonal', 'Tied', 'Full']
(
    df.loc[(df['model_type'] == 'HMM')]
    .groupby(['preprocessing.num_gesture_classes', 'hmm.covariance_type'])
    [['val.macro avg.f1-score', 'val.macro avg.precision', 'val.macro avg.recall']]
    .agg(['count', 'median', 'mean', 'min', 'max', 'std'])
    .T
    .round(3)
)

In [ ]:
model_type = 'HMM'
color='tab:red'
n_classes = ('5', '50', '51',)
for num_gesture_classes in n_classes:
    data = df[
        (df['preprocessing.num_gesture_classes'] == num_gesture_classes)
        & (df['model_type'] == model_type)
    ]

    data['hmm.covariance_type'] = data['hmm.covariance_type'].replace({
        'spherical': 'Spherical',
        'diag': 'Diagonal',
        'full': 'Full',
        'tied': 'Tied',
    })

    fig, axs = plt.subplots(1, 2, figsize=(WIDTH, WIDTH*0.5))
    p_min = data['val.macro avg.precision'].min() # / 1.10
    p_max = data['val.macro avg.precision'].max() # * 1.10
    r_min = data['val.macro avg.recall'].min()    # / 1.10
    r_max = data['val.macro avg.recall'].max()    # * 1.10

    precision_grid, recall_grid = np.meshgrid(
        np.linspace(p_min, p_max, 50), 
        np.linspace(r_min, r_max, 50),
    )

    f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

#     contours = axs[0].contour(
#         precision_grid,
#         recall_grid, 
#         f1_score,
#         levels=np.linspace(
#             f1_score.min(),
#             f1_score.max(),
#             10,
#         ), 
#         colors='black',
#         alpha=0.25
#     )
#     axs[0].clabel(contours, inline=True, fontsize=8, fmt='%.2f')

    sns.scatterplot(
        data=data,
        y='val.macro avg.recall',
        x='val.macro avg.precision',
#         hue='val.macro avg.f1-score',
        hue='hmm.covariance_type',
        hue_order=hue_order,
        alpha=0.5,
        s=10,
        ax=axs[0],
        palette=other_colours,
        edgecolor=None,
    )

    p_range = p_max - p_min
    r_range = r_max - r_min
    axs[0].set(
        title=f"Precision vs Recall"
        f"\n({num_gesture_classes}-class {model_type}s)",
        xlabel='Precision',
        ylabel='Recall',
#         xlim=(0, .05),
#         ylim=(0, 1),
        xlim=(p_min - p_range*0.025, p_max + p_range*0.025),
        ylim=(r_min - r_range*0.025, r_max + r_range*0.025),
    )
    axs[0].legend().set_title('Covariance')
    
    
    sns.stripplot(
        data=data,
        y='val.macro avg.f1-score',
        x='hmm.covariance_type',
        order=hue_order,
        size=3,
        alpha=0.5,
        ax=axs[1],
        palette=other_colours,
#         hue='val.macro avg.f1-score',
        hue='hmm.covariance_type',
        hue_order=hue_order,

    )
    axs[1].set(
        title=f"$F_1$-score vs Covariance Type"
        f"\n({num_gesture_classes}-class {model_type}s)",
        xlabel='Covariance Type',
        ylabel='$F_1$-score',
#         xlim=(p_min - p_range*0.025, p_max + p_range*0.025),
#         ylim=(-0.05, 1.05),
    )
    axs[0].legend().set_title('Covariance Type')
    plt.tight_layout()

#     plt.savefig(
#         f'../../report/src/imgs/graphs/05_in_depth_hmm_{num_gesture_classes}_p_vs_r_covar_type.pdf',
#         bbox_inches='tight',
#     )
    plt.show()

In [ ]:
sns.scatterplot(
    data=df.assign(**{
        'calc_f1-score': lambda ddf: 2 * (ddf['val.macro avg.precision'] * ddf['val.macro avg.precision']) / (ddf['val.macro avg.recall'] + ddf['val.macro avg.recall'])
    }),
    x='val.macro avg.f1-score',
    y='calc_f1-score',
    hue='preprocessing.num_gesture_classes',
    size=.5,
    alpha=0.5,
    edgecolor=None,
)

In [ ]:
nclasses_list = ('5', '50', '51')
for nclasses in nclasses_list:
    fig, ax = plt.subplots(1, 1, figsize=(WIDTH*0.5, WIDTH*0.5))

    data = df[
        (df['preprocessing.num_gesture_classes'] == nclasses)
        & (df['model_type'] == 'HMM')
    ]

    data['val.pred_time_per_obs.log10'] = np.log10(data['val.pred_time_per_obs'])
    data['trn.pred_time_per_obs.log10'] = np.log10(data['trn.pred_time_per_obs'])
    data['fit_time_per_obs.log10'] = np.log10(data['fit_time_per_obs'])

    data['hmm.covariance_type'] = data['hmm.covariance_type'].replace({
        'spherical': 'Spherical',
        'diag': 'Diagonal',
        'full': 'Full',
        'tied': 'Tied',
    })


    sns.scatterplot(
        data=data,
        x='fit_time_per_obs.log10',
        y='trn.pred_time_per_obs.log10',
        s=5,
        alpha=0.5,
        hue='hmm.covariance_type',
        hue_order=hue_order,
        ax=ax,
        palette=other_colours,
        edgecolor=None,
    )

    ax.legend().set_title('Covariance Type')

    ax.set(
        title=f'Fitting time vs inference time\n({nclasses}-class HMMs)',
        xlabel=r'Fit time ($\log_{10}(s/obs)$)',
        ylabel='Inference time ($\log_{10}(s/obs)$)',
    )

    plt.tight_layout()

    plt.savefig(
        f'../../report/src/imgs/graphs/05_in_depth_hmm_inf_trn_time_classes{nclasses}.pdf',
        bbox_inches='tight'
    )

In [ ]:
conf_mats.keys()

In [ ]:
nclasses_list = ('5', '50', '51')
for nclasses in nclasses_list:

    data = df[
        (df['preprocessing.num_gesture_classes'] == nclasses)
        & (df['model_type'] == 'HMM')
    ]
    data['hmm.covariance_type'] = data['hmm.covariance_type'].replace({
        'spherical': 'Spherical',
        'diag': 'Diagonal',
        'full': 'Full',
        'tied': 'Tied',
    })

    conf_mats = {}
    conf_mat_totals = {}
    precisions = {}
    recalls = {}
    f1_scores = {}
    y_preds_dict = {}
    y_true_dict = {}
    reports_dict = {}

    hpar = 'hmm.covariance_type'

    assert len(data[hpar].unique()) < 10

    for i, row in data.iterrows():
        y_true, y_pred = get_npz_data_from_model('../' + row['model_dir'])
        precision, recall, f1_score, _support = sklearn.metrics.precision_recall_fscore_support(
            y_true.flatten(), 
            y_pred.flatten(),
    #         average='macro',
            zero_division=0,
        )

        cm = tf.math.confusion_matrix(
            y_true.flatten(), 
            y_pred.flatten()
        ).numpy()
        hpar_item = row[hpar]

        if hpar_item in conf_mats:
            conf_mats[hpar_item] += cm.astype(float)
            conf_mat_totals[hpar_item] += 1.0
            precisions[hpar_item] += precision
            recalls[hpar_item] += recall
            f1_scores[hpar_item] += f1_score
            y_preds_dict[hpar_item] = np.concatenate((y_preds_dict[hpar_item], y_pred.flatten()))
            y_true_dict[hpar_item] = np.concatenate((y_true_dict[hpar_item], y_true.flatten()))
        else:
            conf_mats[hpar_item] = cm.astype(float)
            conf_mat_totals[hpar_item] = 1.0
            precisions[hpar_item] = precision
            recalls[hpar_item] = recall
            f1_scores[hpar_item] = f1_score
            y_preds_dict[hpar_item] = np.copy(y_pred.flatten())
            y_true_dict[hpar_item] = np.copy(y_true.flatten())

    fig, axs = plt.subplots(
        2, 2,
        figsize=(WIDTH, WIDTH),
        squeeze=False,
    )
    axs = axs.flatten()
    hpar_items = ['Spherical', 'Diagonal', 'Tied', 'Full']
    for i, hpar_item in enumerate(hpar_items):
        conf_mat = conf_mats[hpar_item]
        vis.conf_mat(conf_mat, ax=axs[i])
    #     vis.conf_mat(conf_mat / conf_mat.max(), ax=axs[i], norm=None)
        axs[i].set_title(
            f'{hpar_item.title()} {nclasses}-class HMMs'
    #         f'(Mean of {int(conf_mat_totals[hpar_item])} confusion matrices)'
        )

    plt.tight_layout()

    plt.savefig(
        f'../../report/src/imgs/graphs/05_in_depth_hmm_conf_mats_cov_type_classes{nclasses}.pdf',
        bbox_inches='tight',
    )
    plt.show()

    fig, axs = plt.subplots(
        2, 2,
        figsize=(WIDTH, WIDTH*.4),
    )
    axs = axs.flatten()
    hpar_items = ['Spherical', 'Diagonal', 'Tied', 'Full']
    for i, hpar_item in enumerate(hpar_items):
        conf_mat = conf_mats[hpar_item]
    #     axs[i].set_aspect(1)
        vis.precision_recall_f1(
            precision=precisions[hpar_item] / conf_mat_totals[hpar_item],
            recall=recalls[hpar_item] / conf_mat_totals[hpar_item],
            f1=f1_scores[hpar_item] / conf_mat_totals[hpar_item],
            ax=axs[i],
        )
        axs[i].set_title(f'{hpar_item} {nclasses}-class HMMs')
        if i % 2 != 0:
            axs[i].set_yticks([])
    plt.tight_layout()

    plt.savefig(
        f'../../report/src/imgs/graphs/05_in_depth_hmm_prf1_plots_conv_type_classes{nclasses}.pdf',
        bbox_inches='tight',
    )
    plt.show()

## FFNN Plots

### Heatmap-based pairplot of FFNN Hyperparameters

In [ ]:
data = df[
    (df['model_type'] == 'FFNN') &
    (df['preprocessing.num_gesture_classes'] == '51')
]

hpars_scale = [
    ('nn.learning_rate',       'log10'),
    ('ffnn.nodes_per_layer.1', 'log10'),
    ('ffnn.l2_coefficient',    'log10'),
    ('ffnn.dropout_rate',      'linear'),
]
# x_var_idx = 0
# y_var_idx = 1
def contour_nicely(x, y, z, xlabel, ylabel, xscale, yscale, fig, ax, levels=8):

    ax.tricontour(x, y, z, levels=levels, linewidths=0.25, colors='k')
    cntr2 = ax.tricontourf(x, y, z, levels=levels, cmap=palette)

    fig.colorbar(cntr2, ax=ax)
    ax.scatter(x, y, color='white', s=1)

    ax.set_xlabel(f'{xlabel} ({xscale})')
    if xscale == 'log10':
        ax.set_xticks(ax.get_xticks())
        ax.set_xticklabels([f'{np.power(10, t):.3g}' for t in ax.get_xticks()])

    ax.set_ylabel(f'{ylabel} ({yscale})')
    if yscale == 'log10':
        ax.set_yticks(ax.get_yticks())
        ax.set_yticklabels([f'{np.power(10, t):.3g}' for t in ax.get_yticks()])

    ax.set_title(f'{xlabel} vs {ylabel}')

    
fig, axs = plt.subplots(
    len(hpars_scale), 
    len(hpars_scale), 
    dpi=200, 
    squeeze=False,
    figsize=(WIDTH, WIDTH)
)
z = data['val.macro avg.f1-score'].values
for x_var_idx in range(len(hpars_scale)):
    for y_var_idx in range(x_var_idx+1, len(hpars_scale)):
    
        x = data[hpars_scale[x_var_idx][0]].values
        if hpars_scale[x_var_idx][1] == 'log10':
            x = np.log10(x)
        elif hpars_scale[x_var_idx][1] != 'linear':
            raise NotImplementedError(f"Scale {hpars_scale[x_var_idx][0]} is not implemented")
        y = data[hpars_scale[y_var_idx][0]].values
        if hpars_scale[y_var_idx][1] == 'log10':
            y = np.log10(y)
        elif hpars_scale[y_var_idx][1] != 'linear':
            raise NotImplementedError(f"Scale {hpars_scale[y_var_idx][0]} is not implemented")
            
        contour_nicely(
            y, x, z,
            hpars_scale[y_var_idx][0],
            hpars_scale[x_var_idx][0],
            hpars_scale[y_var_idx][1],
            hpars_scale[x_var_idx][1],
            fig, axs[x_var_idx, y_var_idx]
        )
plt.tight_layout()
plt.show()


## Inference times vs num classes

In [ ]:
df['val.pred_time_per_obs'] = df['val.pred_time'] / df['val.num_observations']
df['val.pred_time_per_obs.log10'] = np.log10(df['val.pred_time_per_obs'].fillna(0))

fig, axs = plt.subplots(2, 2, figsize=(WIDTH, WIDTH))
axs = axs.flatten()
model_types = [m for m in model_colours.keys() if m != 'HFFNN']
for ax, model_type in zip(axs, sorted(model_types)):
    sns.stripplot(
        data=df[
            df['model_type'] == model_type
        ].sort_values('preprocessing.num_gesture_classes'),
        x='preprocessing.num_gesture_classes',
        y='val.pred_time_per_obs',
        hue='model_type',
#         dodge=True,
        s=2,
        legend=False,
        hue_order=list(model_colours.keys()),
        alpha=0.5,
        ax=ax,
    )
    ax.set(
        title=f'{model_type} inference time vs number of classes',
        xlabel='Number of Classes',
        ylabel='Inference time (seconds/observation)',
    )

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_inf_time_vs_num_classes.pdf',
    bbox_inches='tight',
)

## Example Confusion Matrices from baseline models

TODO: Also include models with perfect precision / perfect recall / perfect precision for non-g255 gestures / perfect precision for g255 / perfect recall for non-g255 gestures / perfect recall for g255

In [ ]:
# Load in the dataset/classifier
(
    X_trn, X_val, y_trn, y_val, dt_trn, dt_val
) = common.read_and_split_from_npz("../gesture_data/trn_20_10.npz")

### Plot a model that only predicts the non-gesture class

In [ ]:
# A function for plotting precision-recall + confusion matrices
def plt_pr_conf_mat(y_true, y_preds):
    """Given true and predicted labels, 
    create a confusion matrix and a precision-recall plot."""
    fig, axs = plt.subplots(1, 2, figsize=(WIDTH, WIDTH*0.5))
    if len(y_preds.shape) == 1:
        y_preds = np.array([y_preds])
    # Confusion matrix
    # Get the median confusion matrix
    n_classes = len(np.unique(y_true))
    cm_sum = np.zeros((n_classes, n_classes))
    
    for y_pred in y_preds:
        cm_sum += tf.math.confusion_matrix(y_true, y_pred).numpy()
    cm = cm_sum / len(y_preds)
    vis.conf_mat(cm, ax=axs[1])
    
    f1_sum = 0
    for y_pred in y_preds:
        f1_sum += sklearn.metrics.f1_score(y_true, y_pred, average='macro')
    f1 = f1_sum / len(y_preds)

    # Precision-recall plot
    recall_grid, precision_grid = np.meshgrid(
        np.linspace(0, 1, 100), 
        np.linspace(0, 1, 100)
    )
    f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

    contours = axs[0].contour(
        recall_grid, 
        precision_grid,
        f1_score, 
        levels=np.linspace(0.1, 1, 10), 
        colors='black',
        alpha=0.25
    )
    axs[0].clabel(contours, inline=True, fontsize=8, fmt='%.2f')

    ps = []
    rs = []
    for y_pred in y_preds:
        ps.append(sklearn.metrics.precision_score(y_true, y_pred, average='macro', zero_division=0))
        rs.append(sklearn.metrics.recall_score(y_true, y_pred, average='macro', zero_division=0))
    p = np.mean(ps)
    r = np.mean(rs)

    axs[0].scatter(
        ps, rs,
        color='black',
        s=5,
        alpha=0.5,
    )
    axs[0].set_xlabel('Precision')
    axs[0].set_ylabel('Recall')
    axs[0].set_xlim((-0.01, 1.01))
    axs[0].set_ylim((-0.01, 1.01))
    axs[0].plot([0,1], [0,1], color='black', alpha=.1)
    plt.tight_layout()
#     axs[0].set_title(f'Precision-Recall Graph\n(contours indicate the $F_1$-score)')
    return fig, axs

#### Only predicts the non-gesture class

In [ ]:
# Only predicts 50
y_preds = np.full((30, y_trn.shape[0]), 50)
fig, axs = plt_pr_conf_mat(y_trn, y_preds);
plt.savefig('../../report/src/imgs/graphs/05_pr_conf_mat_only_50.pdf', bbox_inches='tight')

f1_scores = []
for y_pred in y_preds:
    f1_scores.append(sklearn.metrics.f1_score(y_trn, y_pred, average='macro', zero_division=0))
f1_scores = np.array(f1_scores)
print(f1_scores.min(), f1_scores.mean(), f1_scores.std(), f1_score.max())


#### Completely random predictions

In [ ]:
# Predicts randomly
y_preds = np.random.randint(0, 51, size=(30, y_trn.shape[0]))
fig, axs = plt_pr_conf_mat(y_trn, y_preds);
plt.savefig('../../report/src/imgs/graphs/05_pr_conf_mat_random_preds.pdf', bbox_inches='tight')

f1_scores = []
for y_pred in y_preds:
    f1_scores.append(sklearn.metrics.f1_score(y_trn, y_pred, average='macro', zero_division=0))
f1_scores = np.array(f1_scores)
print(f1_scores.min(), f1_scores.mean(), f1_scores.std(), f1_score.max())


#### Wrong orientation

In [ ]:
# Predicts wrong orientation
y_trn_repeated = np.repeat(y_trn[np.newaxis, :], 30, axis=0)
y_preds = np.where(
    # If the gesture is not g255
    y_trn_repeated != 50, 
    # Add/subtract some random multiple of 10
    y_trn_repeated + 10*np.random.randint(
        -(y_trn_repeated // 10), 
        +(5 - y_trn_repeated // 10), 
        y_trn_repeated.shape
    ), 
    # Else do nothing
    y_trn_repeated
)
fig, axs = plt_pr_conf_mat(y_trn, y_preds);
plt.savefig('../../report/src/imgs/graphs/05_pr_conf_mat_wrong_orientation.pdf', bbox_inches='tight')

f1_scores = []
for y_pred in y_preds:
    f1_scores.append(sklearn.metrics.f1_score(y_trn, y_pred, average='macro', zero_division=0))
f1_scores = np.array(f1_scores)
print(f1_scores.min(), f1_scores.mean(), f1_scores.std(), f1_score.max())


#### Wrong finger

In [ ]:
# Predicts wrong finger
y_trn_repeated = np.repeat(y_trn[np.newaxis, :], 30, axis=0)
y_preds = np.where(
    # If the gesture is not g255
    y_trn_repeated != 50, 
    # Then change only the last digit
    y_trn_repeated + np.random.randint(
        -np.mod(y_trn_repeated, 10), 
        +(10 - np.mod(y_trn_repeated, 10)),
        y_trn_repeated.shape
    ), 
    # else keep it the same
    y_trn_repeated
)

fig, axs = plt_pr_conf_mat(y_trn, y_preds);
plt.savefig('../../report/src/imgs/graphs/05_pr_conf_mat_wrong_finger.pdf', bbox_inches='tight')


f1_scores = []
for y_pred in y_preds:
    f1_scores.append(sklearn.metrics.f1_score(y_trn, y_pred, average='macro', zero_division=0))
f1_scores = np.array(f1_scores)
print(f1_scores.min(), f1_scores.mean(), f1_scores.std(), f1_score.max())


In [ ]:
# Predicts wrong finger (but correct hand)
y_trn_repeated = np.repeat(y_trn[np.newaxis, :], 30, axis=0)

y_preds = np.where(
    # If the gesture is not g255
    y_trn_repeated != 50, 
    # Then change only the last digit
    y_trn_repeated + np.random.randint(
        -np.mod(y_trn_repeated, 5), 
        +(5 - np.mod(y_trn_repeated, 5)),
        y_trn_repeated.shape
    ), 
    # else keep it the same
    y_trn_repeated
)

fig, axs = plt_pr_conf_mat(y_trn, y_preds);
plt.savefig('../../report/src/imgs/graphs/05_pr_conf_mat_wrong_finger_correct_hand.pdf', bbox_inches='tight')


f1_scores = []
for y_pred in y_preds:
    f1_scores.append(sklearn.metrics.f1_score(y_trn, y_pred, average='macro', zero_division=0))
f1_scores = np.array(f1_scores)
print(f1_scores.min(), f1_scores.mean(), f1_scores.std(), f1_score.max())


#### Predict 50 as a random gesture

In [ ]:
# Predicts 50 as random gesture
y_trn_repeated = np.repeat(y_trn[np.newaxis, :], 30, axis=0)
y_preds = np.where(
    # If the gesture IS g255
    y_trn_repeated == 50, 
    # Then choose a random non-g255 gesture
    np.random.randint(0, 50, y_trn_repeated.shape), 
    # else keep it the same
    y_trn_repeated
)
fig, axs = plt_pr_conf_mat(y_trn, y_preds);
plt.savefig('../../report/src/imgs/graphs/05_pr_conf_mat_no_gesture_50.pdf', bbox_inches='tight')


f1_scores = []
for y_pred in y_preds:
    f1_scores.append(sklearn.metrics.f1_score(y_trn, y_pred, average='macro', zero_division=0))
f1_scores = np.array(f1_scores)
print(f1_scores.min(), f1_scores.mean(), f1_scores.std(), f1_score.max())


#### High recall model

In [ ]:
# Predicts 50 as random gesture
y_trn_repeated = np.repeat(y_trn[np.newaxis, :], 30, axis=0)
y_preds = np.where(
    # If the gesture IS g255
    y_trn_repeated == 50, 
    # Then choose a random non-g255 gesture
    np.random.randint(0, 51, y_trn_repeated.shape), 
    # else keep it the same
    y_trn_repeated
)
fig, axs = plt_pr_conf_mat(y_trn, y_preds);
plt.savefig('../../report/src/imgs/graphs/05_pr_conf_mat_high_recall.pdf', bbox_inches='tight')

f1_scores = []
for y_pred in y_preds:
    f1_scores.append(sklearn.metrics.f1_score(y_trn, y_pred, average='macro', zero_division=0))
f1_scores = np.array(f1_scores)
print(f1_scores.min(), f1_scores.mean(), f1_scores.std(), f1_score.max())


#### High precision model

In [ ]:
y_trn_repeated = np.repeat(y_trn[np.newaxis, :], 5, axis=0)
y_preds = np.where(
    y_trn_repeated != 50,
    50,
    y_trn_repeated,
)

# np.clip(np.random.randint(-1, 1, size=y_trn_repeated.shape) + y_trn_repeated, 0, 50)
fig, axs = plt_pr_conf_mat(y_trn, y_preds);
# plt.savefig('../../report/src/imgs/graphs/05_pr_conf_mat_high_precision.pdf', bbox_inches='tight')

# f1_scores = []
# for y_pred in y_preds:
#     f1_scores.append(sklearn.metrics.f1_score(y_trn, y_pred, average='macro', zero_division=0))
# f1_scores = np.array(f1_scores)
# print(f1_scores.min(), f1_scores.mean(), f1_scores.std(), f1_score.max())


### Finally, plot all confusion matrices together

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(WIDTH, WIDTH*2/3))
axs = axs.flatten()

# Random model:
y_pred = np.random.randint(0, 51, y_trn.shape)
cm_val = tf.math.confusion_matrix(y_trn, y_pred).numpy()
vis.conf_mat(cm_val, ax=axs[0])
f1 = sklearn.metrics.f1_score(y_trn, y_pred, average='macro')
axs[0].set_title(f"Random model\n$F_1$={f1:.4}")

# Only predicts 50
y_pred = np.full(y_trn.shape, 50)
cm_val = tf.math.confusion_matrix(y_trn, y_pred).numpy()
vis.conf_mat(cm_val, ax=axs[1])
f1 = sklearn.metrics.f1_score(y_trn, y_pred, average='macro')
axs[1].set_title(f"Only predicts 50\n$F_1$={f1:.4}")

# Perfect, but random orientation:
y_pred = np.where(
    # If the gesture is not g255
    y_trn != 50, 
    # Add/subtract some random multiple of 10
    y_trn + 10*np.random.randint(
        -(y_trn // 10), 
        +(5 - y_trn // 10), 
        y_trn.shape
    ), 
    # Else do nothing
    y_trn
)
cm_val = tf.math.confusion_matrix(y_trn, y_pred).numpy()
vis.conf_mat(cm_val, ax=axs[2])
f1 = sklearn.metrics.f1_score(y_trn, y_pred, average='macro')
axs[2].set_title(f"Perfect,\nbut random orientation\n$F_1$={f1:.4}")

# Perfect, but random finger:
y_pred = np.where(
    # If the gesture is not g255
    y_trn != 50, 
    # Then change only the last digit
    y_trn + np.random.randint(
        -np.mod(y_trn, 10), 
        +(10 - np.mod(y_trn, 10)),
        y_trn.shape
    ), 
    # else keep it the same
    y_trn
)
cm_val = tf.math.confusion_matrix(y_trn, y_pred).numpy()
vis.conf_mat(cm_val, ax=axs[3])
f1 = sklearn.metrics.f1_score(y_trn, y_pred, average='macro')
axs[3].set_title(f"Perfect,\nbut random finger\n$F_1$={f1:.4}")

# Perfect, but random finger (same hand):
y_pred = np.where(
    # If the gesture is not g255
    y_trn != 50, 
    # Then change only the last digit
    y_trn + np.random.randint(
        -np.mod(y_trn, 5), 
        +(5 - np.mod(y_trn, 5)),
        y_trn.shape
    ), 
    # else keep it the same
    y_trn
)
cm_val = tf.math.confusion_matrix(y_trn, y_pred).numpy()
vis.conf_mat(cm_val, ax=axs[4])
f1 = sklearn.metrics.f1_score(y_trn, y_pred, average='macro')
axs[4].set_title(f"Perfect,\nbut random finger on the correct hand\n$F_1$={f1:.4}")

# Never predicts 50
y_pred = np.where(
    # If the gesture IS g255
    y_trn == 50, 
    # Then choose a random non-g255 gesture
    np.random.randint(0, 50, y_trn.shape), 
    # else keep it the same
    y_trn
)
cm_val = tf.math.confusion_matrix(
    y_trn, 
    y_pred,
).numpy()
vis.conf_mat(cm_val, ax=axs[5])
f1 = sklearn.metrics.f1_score(y_trn, y_pred, average='macro')
axs[5].set_title(f"Perfect,\nbut predicts 50 as a random gesture\n$F_1$={f1:.4}")

plt.savefig('../../report/src/imgs/graphs/05_example_conf_mats.pdf')
plt.tight_layout()
plt.show()

## PCA Plots

### PCA decomposition only including the gesture classes

In [ ]:
(
    X_trn, X_val, y_trn, y_val, dt_trn, dt_val
) = common.read_and_split_from_npz("../gesture_data/trn_20_10.npz")


In [ ]:
%%script false --no-raise-error
from sklearn.manifold import TSNE

shape = (np.nonzero(y_trn != 50)[0].shape[0] * 5,)
mask = np.concatenate((
    np.random.choice(np.nonzero(y_trn == 50)[0], shape), 
    np.nonzero(y_trn != 50)[0]
))

# fig, axs = plt.subplots(1, 3, figsize=(WIDTH, WIDTH/3), dpi=200)
fig, axs = plt.subplots(1, 3, figsize=(30, 20), dpi=200)
for i, perplexity in enumerate([35, 45, 55]):
    print(f"PERPLEXITY: {perplexity}")
    X_embedded = TSNE(
        n_components=2,
        n_iter=1500,
        learning_rate='auto',
        init='random',
        perplexity=perplexity,
        verbose=True
    ).fit_transform(X_trn[mask].reshape((X_trn[mask].shape[0], 600)))

    hues = np.array([ "0$^\circ$", "45$^\circ$", "90$^\circ$", "135$^\circ$", "180$^\circ$", "Non-gesture"])
    styles = np.array([ "L5", "L4", "L3", "L2", "L1", "R1", "R2", "R3", "R4", "R5", "Non-gesture"])
#     fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH), dpi=200)
    argsort = np.argsort(y_trn[mask])

    sns.scatterplot(
        x=X_embedded[:, 0][argsort],
        y=X_embedded[:, 1][argsort],
        hue=hues[(y_trn[mask][argsort] // 10)],
        style=styles[(y_trn[mask][argsort] % 10)],
        s=10,
        alpha=0.5,
        ax=axs.flatten()[i],
        legend=False,
        edgecolor=None,
    )
    axs.flatten()[i].set_title(f"Perplexity: {perplexity}")

In [ ]:
def move_legend(from_ax, to_ax, title=None):
    # Fetch the current legend
    handles, labels = from_ax.get_legend_handles_labels()
    title = title if title is not None else from_ax.legend().get_title()
    # Remove it from the ax
    from_ax.legend().remove()
    # Add it to the new ax with some styling
    to_ax.legend(
        handles=handles, 
        labels=labels,
        loc='center',
        fancybox=False, 
        edgecolor="black",
        title=title
#         linewidth=0.5,
    )
    # Remove the axis of the ax
    to_ax.axis('off')
    return from_ax, to_ax

In [ ]:
print("Executing PCA")
pca = PCA(n_components=2)
X_reshaped = X_trn.reshape((X_trn.shape[0], 600))
X_tfrm = pca.fit_transform(X_reshaped)
print("PCA complete")

In [ ]:
from sklearn.decomposition import PCA

fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH), dpi=200)
colours = np.array([ "tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple", 'k'])
# markers = np.array(['o', 'x', 's', '+', 'D', 'p', '^', 'd', 'v', '*', 'x'])

# fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH), dpi=300)
# mask = (y_trn == 50)
# axs[1].scatter(
#     X_tfrm[:, 0][mask],
#     X_tfrm[:, 1][mask],
#     color='black',
#     alpha=0.1,
#     s=5,
#     edgecolor='none',
# )
for gidx in range(0, 51):
    ax.scatter(
        X_tfrm[:, 0][y_trn == gidx],
        X_tfrm[:, 1][y_trn == gidx],
        color=colours[gidx // 10],
        marker=markers[gidx % 10],
        alpha=0.75 if gidx != 50 else 0.1,
        zorder=10 if gidx != 50 else 0,
        s=10 if gidx != 50 else 5,
        edgecolor='none',
    )
ax.set_title('PCA plot of the training data\n(class 50 in black)')

ax.set(
    xlabel='Principal Component 1',
    ylabel='Principal Component 2',
)

plt.tight_layout()
print("TODO: Also add a legend here")

plt.savefig(
    '../../report/src/imgs/graphs/05_pca_plot.png',
    bbox_inches='tight'
)

In [ ]:
n = 11
markers = [
    "o", "X", (4, 0, 45), "P", (4, 0, 0), (4, 1, 0), "^", (4, 1, 45), "v",
]

# Now generate more from regular polygons of increasing order
s = 5
while len(markers) < n:
    a = 360 / (s + 1) / 2
    markers.extend([(s + 1, 1, a), (s + 1, 0, a), (s, 1, 0), (s, 0, 0)])
    s += 1

markers = np.array([m for m in markers[:n]])
markers

### PCA plot showing just an interesting subset

In [ ]:
pca = PCA(n_components=2)
X_reshaped = X_trn.reshape((X_trn.shape[0], 600))
X_tfrm = pca.fit_transform(X_reshaped)

In [ ]:
colours = np.array([ "tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple"])

@interact(
    x_start=(-1500, 1500, 50),
    y_start=(-1500, 1500, 50),
    x_length=(-1500, 1500, 50),
    y_length=(-1500, 1500, 50),
)
def fn(x_start=-150, y_start=1000, x_length=500, y_length=500):
    x_finsh = x_start + x_length
    y_finsh = y_start + y_length

    fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=100)

    selection_mask = (
        (x_start <= X_tfrm[:, 0]) & (X_tfrm[:, 0] <= x_finsh) &
        (y_start <= X_tfrm[:, 1]) & (X_tfrm[:, 1] <= y_finsh)
    )
    X_subset = X_tfrm[selection_mask]
    y_subset = y_trn[selection_mask]
#     ax.scatter(
#         X_subset[:, 0],
#         X_subset[:, 1],
#         c='black',
#         alpha=0.1,
#     )
    
    y_mask = (y_trn == 50)
    ax.scatter(
        X_subset[:, 0][y_subset == 50],
        X_subset[:, 1][y_subset == 50],
        color='black',
        alpha=0.1,
        s=20,
        edgecolor='none',
    )
    ax.scatter(
        X_subset[:, 0][y_subset != 50],
        X_subset[:, 1][y_subset != 50],
        color=colours[(y_subset[y_subset != 50] // 10)],
        alpha=0.75,
#         s=5,
        edgecolor='none',
    )


### PCA plot that connects sequential datapoints

In [ ]:
pca = PCA(n_components=2)
X_reshaped = X_trn.reshape((X_trn.shape[0], 600))
X_tfrm = pca.fit_transform(X_reshaped)

order = np.argsort(dt_trn)
X_tfrm = X_tfrm[order]

In [ ]:


limit = 1000
@interact(start=(0, len(X_tfrm), 25), length=(0, len(X_tfrm), 50))
def fn(start=0, length=500):
    finsh = min(start + length, len(X_tfrm))
    fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH), dpi=100)
    ax.plot(
        X_tfrm[:, 0][start:finsh],
        X_tfrm[:, 1][start:finsh],
        zorder=0,
        c='black',
        alpha=0.1,
    )
    sns.scatterplot(
        x=X_tfrm[:, 0][start:finsh],
        y=X_tfrm[:, 1][start:finsh],
        hue=dt_trn[order][start:finsh],
        legend=False,
        s=(10 + 90*(y_trn != 50)[order][start:finsh]),
        edgecolor=np.where((y_trn != 50)[order][start:finsh], 'black', 'none'),
        linewidth=.5,
        edgecolor=None,
    )
    
    idxs = np.nonzero(y_trn[order][start:finsh] != 50)[0]
    for idx in idxs:
        ax.text(
            X_tfrm[start:finsh][idx, 0],
            X_tfrm[start:finsh][idx, 1],
            y_trn[order][start:finsh][idx],
            va='center',
            ha='center',
        )
#     ax.set_xlim((
#         X_tfrm[:, 0].min() / 1.1,
#         X_tfrm[:, 0].max() * 1.1,
#     ))
#     ax.set_ylim((
#         X_tfrm[:, 1].min() / 1.1,
#         X_tfrm[:, 1].max() * 1.1,
#     ))

### PCA plot with ellipses

In [ ]:
pca = PCA(n_components=2)
X_reshaped = X_trn.reshape((X_trn.shape[0], 600))
X_tfrm = pca.fit_transform(X_reshaped[y_trn != 50])
y_tfrm = y_trn[y_trn != 50]

In [ ]:
# https://matplotlib.org/stable/gallery/statistics/confidence_ellipse.html
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1] / np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensional dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse(
        (0, 0), 
        width=ell_radius_x * 2, 
        height=ell_radius_y * 2,
        facecolor=facecolor, 
        **kwargs,
    )

    # Calculating the standard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the standard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)
    
#     print(f"{scale_x=}, {scale_y=}\n{mean_x=}, {mean_y=}")

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    
    return ax.add_patch(ellipse)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH))
# gidx = 14

hues = np.array([ "0$^\circ$", "45$^\circ$", "90$^\circ$", "135$^\circ$", "180$^\circ$" ])
styles = np.array([ "L5", "L4", "L3", "L2", "L1", "R1", "R2", "R3", "R4", "R5" ])
markers = ['o', 's', 'D', '^', 'v', '>', '<', 'p', 'H', '+']
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple']

for gidx in range(50):
    ax.scatter(
        X_tfrm[y_tfrm == gidx, 0],
        X_tfrm[y_tfrm == gidx, 1],
        color=colors[gidx // 10],
        marker=markers[gidx % 10],
        s=5,
        alpha=0.5,
        label=gidx
    )
    confidence_ellipse(
        X_tfrm[y_tfrm == gidx, 0],
        X_tfrm[y_tfrm == gidx, 1],
        ax,
        n_std=2,
        edgecolor=colors[gidx // 10],
        alpha=0.5,
    )
plt.legend()

## Visualise mis-predictions

1. Load in a continuous dataset
2. Load in a classifier
3. Use the classifier to make predictions on the dataset
4. Visualise the mispredictions, but *with context*

### Load in a model for which to evaluate the mis-predictions

In [ ]:
# Load in the dataset/classifier
(
    X_trn, X_val, y_trn, y_val, dt_trn, dt_val
) = common.read_and_split_from_npz("../gesture_data/trn_20_10.npz")

clf = models.load_tf('../src/saved_models/ffnn_2023-09-18T14:05:16.363404')
const = common.read_constants('../src/constants.yaml')
sensor_names = list(const['sensors'].values())

### Visualise True and Mispredicted gestures

Plot all the observations which have the ground truth being gesture 255 but the model is not predicting g255

In [ ]:
y_pred = clf.predict(X_val)

for gidx in np.unique(y_val):
    if gidx == 50: continue
    pred_indxs = np.nonzero((y_val == 50) & (y_pred == gidx))[0]
    true_indxs = np.nonzero(y_val == gidx)[0]
    axs = vis.cmp_ts(
        X_val[true_indxs],
    )
    vis.cmp_ts(
        X_val[pred_indxs],
        color='tab:red',
        axs=axs,
    )

#     distances = np.abs(true_indxs[:, np.newaxis] - pred_indxs).min(axis=0)

    plt.suptitle(f'Model predicted {gidx}, ground truth: 50 \
                 \nGesture {gidx} in grey, mispredicted in red ({len(pred_indxs)} observations) \
                 \nindices: {pred_indxs}')
    plt.tight_layout()
#     plt.savefig(f'../src/notebooks/pred_{gidx:0>2}_truth_50.pdf', bbox_inches='tight')
    plt.show()
#     if gidx > 5:
    break


## Eval model on test set

In [ ]:
model_dir = '../src/saved_models/ffnn_2023-10-08T14:47:07.901649'
tst_data = np.load('../gesture_data/tst_20_10.npz')
y_tst = tst_data['y_tst']
X_tst = tst_data['X_tst']
dt_tst = tst_data['dt_tst']
clf = models.load_tf(model_dir)

In [ ]:
model_type = clf.config['model_type']
                                                                         
y_pred = clf.predict(X_tst)
clf_report_dict = sklearn.metrics.classification_report(
    y_tst.astype(int),
    y_pred.astype(int),
    output_dict=True,
    zero_division=0,
)
clf_report = pd.json_normalize(clf_report_dict)
# print(sklearn.metrics.classification_report(
#     y_tst.astype(int),
#     y_pred.astype(int),
#     output_dict=False,
#     zero_division=0,
# ))
                            
f1 = clf_report['macro avg.f1-score'].values[0]
precision = clf_report['macro avg.precision'].values[0]
recall = clf_report['macro avg.recall'].values[0]
# print("val.macro avg.f1-score", f1)
# print("val.macro avg.precision", precision)
# print("val.macro avg.recall", recall)
# print (clf_report['macro avg.f1-score'].values[0])

cm = tf.math.confusion_matrix(y_tst, y_pred, num_classes=51).numpy()
# cm[-1, -1] = 0
fig, axs = plt.subplots(2, 1, figsize=(WIDTH, WIDTH))
vis.conf_mat(
    cm,
    ax=axs[0],
)
axs[0].set_title(
    f'Confusion Matrix (Test set)\n'
    f'$F_1$={np.round(f1, 3)}, Precision={np.round(precision, 3)}, Recall={np.round(recall, 3)}'
)

axs[1].set_aspect(2.5)
vis.precision_recall_f1(
    report=clf_report_dict,
    ax=axs[1],
)

plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_tst_set_conf_mat.pdf'
)

In [ ]:
ddf = pd.DataFrame()
ddf['Recall'] = {
    int(k): d['recall']
    for k, d in clf_report_dict.items()
    if type(d) is dict and k.isdigit()
}
ddf['Precision'] = {
    int(k): d['precision']
    for k, d in clf_report_dict.items()
    if type(d) is dict and k.isdigit()
}
ddf['$F_1$-score'] = {
    int(k): d['f1-score']
    for k, d in clf_report_dict.items()
    if type(d) is dict and k.isdigit()
}
ddf = ddf.reset_index()
ddf

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH))

hues = np.array([ "0$^\circ$", "45$^\circ$", "90$^\circ$", "135$^\circ$", "180$^\circ$", 'Non-Gesture'])
styles = np.array([ "L5", "L4", "L3", "L2", "L1", "R1", "R2", "R3", "R4", "R5", 'Non-Gesture'])

recall_grid, precision_grid = np.meshgrid(
    np.linspace(0, 1, 100), 
    np.linspace(0, 1, 100)
)
f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

contours = ax.contour(
    recall_grid, 
    precision_grid,
    f1_score, 
    levels=np.linspace(0.1, 1, 10), 
    colors='black',
    alpha=0.25
)
ax.clabel(contours, inline=True, fontsize=8, fmt='%.2f')

# fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH))
sns.scatterplot(
    data=ddf,
    x='Recall',
    y='Precision',
    hue=hues[ddf.index // 10],
    style=styles[ddf.index % 10],
    ax=ax,
    edgecolor=None,
)
ax.set(
    xlim=(-0.05, 1.05),
    ylim=(-0.05, 1.05),
)

ax.set(
    title='Precision and Recall for all gestures (best model)'
)
plt.savefig(
    '../../report/src/imgs/graphs/05_p_r_best_model.pdf'
)

# Interactive plot to see data at a certain time

In [ ]:

df = read.read_data(
    '../gesture_data/train/', 
#     constants_path='../src/constants.yaml',
)
df['gidx'] = df['gesture'].apply(lambda g: int(g[-4:]) if g != 'gesture0255' else 50)

@interact(dt='2022-10-08T20:23:46.665276000')
def fn(dt='2022-10-08T20:23:46.665276000'):
    try:
        dt = pd.to_datetime(dt)
    except Exception as e:
        print(e)
        return
#     fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH))
    mask = df['datetime'].between(
        dt - pd.to_timedelta(1, 'second'),
        dt + pd.to_timedelta(1, 'second')
    )
    
    vis.cmp_ts(
        [df.loc[mask, sensor_names].values]
    )


# Plot a CSV file + predictions

In [ ]:
csv_path = '../gesture_data/saved_from_cli_2023-10-08T22:24:06.csv'
model_dir = '../src/saved_models/ffnn_2023-10-08T14:14:07.933452'

import yaml
with open('../gesture_data/gesture_info.yaml', 'r') as f:
    gesture_info = yaml.safe_load(f)['gestures']

sensors = list(common.read_constants('./constants.yaml')["sensors"].values())
df = pd.read_csv(
    csv_path,
    names=["datetime", "gesture"] + sensors,
    parse_dates=["datetime"],
    date_format='ISO8601',
)
df['file'] = csv_path
X, y, dt = common.make_windows(
    df,
    20,
    constants_path='../src/constants.yaml',
    pbar=tqdm.tqdm(total=len(df), desc="Making windows"),
)
clf = models.load_tf(model_dir)
y_pred = clf.predict(X)
y_pred_probs = clf.predict_proba(X)

## Plot data, predictions, keystrokes over time

In [ ]:
# clf=None
@interact(start=(0, len(df), 200), duration=(0, len(df), 200))
def fn(start=10, duration=500):
    fig, axs = plt.subplots(3 if clf is not None else 2, 1, figsize=(WIDTH, WIDTH))
    for i in range(X.shape[-1]):
        axs[0].plot(
            X[start:start+duration, 0, i],
            alpha=0.5,
            c=['tab:red', 'tab:green', 'tab:blue'][i%3],
            lw=1,
        )
    sns.heatmap(
        X[start:start+duration, 0, :].T,
        cmap='Spectral',
        ax=axs[1],
        cbar=False,
        vmin=290,
        vmax=910,
        yticklabels=5,
    )

    if clf is not None:
        sns.heatmap(
            y_pred_probs[start-10:start+duration-10, :].T,
            vmin=0,
            vmax=1,
            ax=axs[2],
            cbar=False,
            yticklabels=5,
            cmap='Spectral',
        )
        axs[2].set(
            ylabel='Predicted\ngesture',
        )
        for idx in np.nonzero(y_pred[start-10:start+duration-10] != 50)[0]:
            gidx = y_pred[start-10:start+duration-10][idx]
            gidx = 255 if gidx == 50 else gidx
            txt = gesture_info[f'gesture{gidx:0>4}']['key']
            axs[0].text(
                idx, 
                800,
                txt,
            )

    plt.subplots_adjust(hspace=0.05)
    
    axs[0].set(
        ylabel='Sensor value',
        xticks=[],
        ylim=(250, 950)
    )
    axs[1].set(
        ylabel='Sensor number',
        xticks=[],
    )
    axs[0].margins(0)
    with_clf =' and model predictions' if clf is not None else ''
    axs[0].set_title(
        f'Sensor values{with_clf} over time from {np.round(start/40, 2)}s\n'
        f'(duration: {np.round(duration/40, 2)} seconds)'
    )


## Easily convert text to gestures

In [ ]:
def char_to_gesture(c):
    c_old = c
    if c == ' ':
        c = 'space'
    for k, v in gesture_info.items():
        if v['key'] == c.lower():
            desc = (
                v["description"]
                .replace("l", "left ")
                .replace("r", "right")
                .replace(" 1", " thumb")
                .replace(" 2", " index")
                .replace(" 3", " middle")
                .replace(" 4", " ring")
                .replace(" 5", " pinky")
            )
            return f'[{c_old}] {k}  {desc}'
text = 'the quick brown fox jumped over the lazy dog'

print('\n'.join(char_to_gesture(c) for c in text))

# Misc Methodology chapter plots

In [ ]:
sns.stripplot(
    data=df,
    x='model_type',
    y='fit_time',
    alpha=0.5,
    s=5,
)
plt.show()

sns.stripplot(
    data=df,
    x='model_type',
    y='val.pred_time',
    alpha=0.5,
    s=5,
)

In [ ]:
subset.groupby('model_type').agg({
    'group_idx': ['nunique', 'count'], 
     'fit_time': 'mean',
     'val.pred_time': 'mean',
})

# ['group_idx'].nunique()

In [ ]:
y_4_true = np.array([
    0, 1, 2, 3, 4, 5,
    0, 1, 2, 3, 4, 5,
    0, 1, 2, 3, 4, 5,
    0, 1, 2, 3, 4, 5,
    0, 1, 2, 3, 4, 5,
])
y_4_pred = np.array([
    0, 1, 2, 3, 4, 5,
    0, 1, 2, 3, 5, 4,
    0, 1, 2, 3, 4, 5,
    2, 2, 2, 2, 2, 2,
    0, 1, 2, 3, 4, 5,

])
conf_mat = tf.math.confusion_matrix(y_4_true, y_4_pred).numpy()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(WIDTH, WIDTH))

titles = np.array([
    ["Count of observations", "Normalized by columns"],
    ["Normalized by rows", "Normalized by total sum"],
])

for i in (0, 1):
    for j in (0, 1):
        if i == j == 0:
            div = 1
        elif i == j == 1:
            div = conf_mat.sum()
        elif i == 0 and j == 1:
            div = conf_mat.sum(axis=0)
        elif i == 1 and j == 0:
            div = conf_mat.sum(axis=1)
        sns.heatmap(
            conf_mat / div,
            square=True,
            annot=True,
            fmt='.0f' if i == j == 0 else '.2f',
            mask=conf_mat == 0,
            cmap='Spectral',
            vmin=0,
            vmax=conf_mat.max() if i == j == 0 else 1.0,
            ax=axs[i, j],
        )
        axs[i, j].set(
            title=titles[i, j],
            xlabel='Predicted',
            ylabel='Ground Truth',
        )
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/04_example_conf_mat.pdf',
    bbox_inches='tight',
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH*.25))
report = sklearn.metrics.classification_report(
    y_4_true,
    y_4_pred,
    zero_division=0,
    output_dict=True,
)
vis.precision_recall_f1(report, ax=ax, annot=True)
# ax.set_xticks([0, 1, 2, 3, 4, 5]);
# ax.set_xticklabels([0, 1, 2, 3, 4, 5]);
ax.set(
    title='Recall, Precision, and $F_1$-score',
    xlabel='Class',
)
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/04_prec_rec_f1_example.pdf',
    bbox_inches='tight',
)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH))

recall_grid, precision_grid = np.meshgrid(
    np.linspace(0, 1, 100), 
    np.linspace(0, 1, 100)
)
f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

contours = ax.contour(
    recall_grid, 
    precision_grid,
    f1_score, 
    levels=np.linspace(0.05, 1, 19), 
    cmap='Spectral'
#     colors='black',
#     alpha=0.5
)
ax.clabel(contours, inline=True, fmt='%.2f')
ax.set(
    title='Precision vs Recall, with $F_1$-score on contours',
    ylabel='Precision',
    xlabel='Recall',
    xticks=np.arange(0, 1.01, 0.1),
    yticks=np.arange(0, 1.01, 0.1),
)
plt.savefig(
    '../../report/src/imgs/graphs/04_precision_recall_f1.pdf'
)

In [ ]:
y_4_true = np.array([
    5, 5, 5, 5, 5, 5, 5, 0, 0, 0,
    5, 5, 5, 5, 5, 5, 5, 1, 1, 1,
    5, 5, 5, 5, 5, 5, 5, 2, 2, 2,
    5, 5, 5, 5, 5, 5, 5, 3, 3, 3,
    5, 5, 5, 5, 5, 5, 5, 4, 4, 4,
])
y_4_pred = np.array([
    5, 5, 5, 5, 5, 5, 0, 5, 0, 0,
    5, 5, 5, 5, 5, 5, 1, 5, 5, 1,
    5, 5, 5, 5, 5, 5, 2, 5, 2, 2,
    5, 5, 5, 5, 5, 5, 3, 5, 4, 3,
    5, 5, 5, 5, 5, 5, 4, 5, 4, 4,

])
conf_mat = tf.math.confusion_matrix(y_4_true, y_4_pred).numpy()

sns.heatmap(
    conf_mat / conf_mat.sum(axis=1),
    square=True,
    annot=True,
#     fmt='.0f' if i == j == 0 else '.2f',
    mask=conf_mat == 0,
    cmap='Spectral',
)
plt.show()
report = sklearn.metrics.classification_report(
    y_4_true,
    y_4_pred,
    zero_division=0,
    output_dict=True,
)
vis.precision_recall_f1(report, annot=True)


# Misc Results Chapter Plots

### Read in all the data

In [ ]:
df = read.read_data(
    '../gesture_data/train/', 
    constants_path='../src/constants.yaml'
)
df['gidx'] = df['gesture'].apply(lambda g: int(g[-4:]) if g != 'gesture0255' else 50)
const = common.read_constants('../src/constants.yaml')
sensor_names = list(const['sensors'].values())

## All observations of gesture XXX

In [ ]:
sensor_names

In [ ]:
def prettify_sensor_name(sensor_name):
    return {
        "l5x": "Left Little X",
        "l5y": "Left Little Y",
        "l5z": "Left Little Z",
        "l4x": "Left Ring X",
        "l4y": "Left Ring Y",
        "l4z": "Left Ring Z",
        "l3x": "Left Middle X",
        "l3y": "Left Middle Y",
        "l3z": "Left Middle Z",
        "l2x": "Left Index X",
        "l2y": "Left Index Y",
        "l2z": "Left Index Z",
        "l1x": "Left Thumb X",
        "l1y": "Left Thumb Y",
        "l1z": "Left Thumb Z",
        "r1x": "Right Thumb X",
        "r1y": "Right Thumb Y",
        "r1z": "Right Thumb Z",
        "r2x": "Right Index X",
        "r2y": "Right Index Y",
        "r2z": "Right Index Z",
        "r3x": "Right Middle X",
        "r3y": "Right Middle Y",
        "r3z": "Right Middle Z",
        "r4x": "Right Ring X",
        "r4y": "Right Ring Y",
        "r4z": "Right Ring Z",
        "r5x": "Right Little X",
        "r5y": "Right Little Y",
        "r5z": "Right Little Z",
    }.get(sensor_name, sensor_name)


for gidx in (0, 5, 11, 16, 22, 27, 33, 38, 44, 49):
    idxs = np.nonzero(df['gidx'] == gidx)[0]
    cross_idxs = idxs[:, np.newaxis] + np.arange(-10, 21)
    fig, axs = plt.subplots(6, 5, figsize=(WIDTH, WIDTH*5/6))
    axs = vis.cmp_ts(
        df[sensor_names].values[cross_idxs],
        axs=axs
    )

    for ax in axs.flatten():
        ax.set_title(
            prettify_sensor_name(ax.get_title()),
            loc='center',
            y=0.85,
        )
        ax.set_ylim((250, 990))

    plt.suptitle(f'Gesture {gidx}')
    plt.tight_layout()
    plt.savefig(
        f'../../report/src/imgs/graphs/05_example_g{gidx:0>4}_plot.pdf'
    )
    plt.show()
# for ax in axs[:, -1]:
#     ax.remove()

## Correlations between the different gestures

In [ ]:
timestep = 0
# Load in the dataset/classifier
(
    X_trn, X_val, y_trn, y_val, dt_trn, dt_val
) = common.read_and_split_from_npz("../gesture_data/trn_20_10.npz")
const = common.read_constants('../src/constants.yaml')
sensor_names = list(const['sensors'].values())
X_data = X_trn[y_trn != 50][:, timestep, :]
y_data = y_trn[y_trn != 50]

In [ ]:
fig, axs = plt.subplots(
    5, 10,
    figsize=(WIDTH, WIDTH*2),
    dpi=200,
)
for i in range(5):
    print(f'gesture {i}_', flush=True)
    for j in range(10):
        sns.heatmap(
            pd.DataFrame(X_data[y_data == (i * 10 + j)]).corr(),
            vmin=-1, 
            vmax=1,
            center=0,
            cbar=False,
            ax=axs[i, j],
            xticklabels=[s.upper() for s in sensor_names],
            yticklabels=[s.upper() for s in sensor_names],
            square=True,
        )
        axs[i, j].set_title(f'Gesture {i * 10 + j}')
plt.subplots_adjust(hspace=0.35, wspace=0.25)

In [ ]:
sns.heatmap(
    pd.DataFrame(X_data).corr(),
    vmin=-1, 
    vmax=1,
    center=0,
#     cbar=False,
    xticklabels=[s.upper() for s in sensor_names],
    yticklabels=[s.upper() for s in sensor_names],
    square=True,
)
plt.xlabel('Sensor')
plt.ylabel('Sensor')
plt.title('Correlations between sensors\n(over all training data)')
plt.savefig(
    '../../report/src/imgs/graphs/05_correlations.pdf',
    bbox_inches='tight'
)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(WIDTH, WIDTH/3))
axis = ['X', 'Y', 'Z']
for i in range(3):
    sns.heatmap(
        pd.DataFrame(X_data[:, i::3]).corr(),
        vmin=-1, 
        vmax=1,
        center=0,
        cbar=False,
        square=True,
        ax=axs[i]
    )
    axs[i].set_title(f'Correlations between {axis[i]}-axis sensors\n(over all training data)')
plt.tight_layout()
plt.show()


## Time-series heatmap + line plots

In [ ]:

def plt_subset(s, f):
    df = read.read_data(
        '../gesture_data/train/', 
        constants_path='../src/constants.yaml'
    )
    df['gidx'] = df['gesture'].apply(lambda g: int(g[-4:]) if g != 'gesture0255' else 50)
    const = common.read_constants('../src/constants.yaml')
    sensor_names = list(const['sensors'].values())
    data = df[sensor_names].values[s:f]
    
    fig, axs = plt.subplots(2, 1, figsize=(WIDTH, WIDTH*.5))
    sns.heatmap(
        data.T,
        ax=axs[0],
        cbar=False
    )

    axs[1].plot(
        data,
        color='black',
        alpha=0.2,
        linewidth=1,
    )
    plt.margins(0)
    plt.show()
# plt_subset(91_000, 95_000)
plt_subset(93_000, 93_200)

## Histogram of class distributions

In [ ]:
df['gidx'].hist()
plt.show()
df.loc[df['gidx'] != 50, 'gidx'].hist()
df['gidx'].value_counts() / len(df['gidx']) * 100

## All observations of one gesture

In [ ]:
gidx = 0
before = 10
after = 10
idxs = np.nonzero(df['gidx'] == gidx)[0][:, np.newaxis] + np.arange(-before, after+1)

vis.cmp_ts(df[sensor_names].values[idxs + 10]);
plt.tight_layout()
plt.show()

In [ ]:
np.arange(-before, after+1)

## 3D plot of the raw acceleration data

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
vals = df[['l5x', 'l5y', 'l5z']].values[:10000]
ax.plot(
    vals[:, 0], 
    vals[:, 1], 
    vals[:, 2], 
    label='3D Line'
)